In [1]:
#!pip install unidecode numpy cython

In [2]:
#!pip install -U biogeme

In [3]:
%load_ext autoreload
%autoreload 2
from IPython.display import clear_output
import sys
sys.path.append('./src')
from utils_ML import *
#from allVariables import *
clear_output(wait=False)

In [4]:
# matplotlib options
plt.style.use('ggplot')
%matplotlib inline
plt.rcParams['figure.figsize'] = (18, 12)

In [5]:
#import dataset
pandas = pd.read_table("http://transp-or.epfl.ch/data/swissmetro.dat")

#rearrange dataset
#return from work = commuter
pandas.loc[:, ('PURPOSE')][(pandas['PURPOSE']==5)] = 1
#return from shopping = shopping
pandas.loc[:, ('PURPOSE')][(pandas['PURPOSE']==6)] = 2
#return from business = business
pandas.loc[:, ('PURPOSE')][(pandas['PURPOSE']==7)] = 3
#return from leisure = leisure
pandas.loc[:, ('PURPOSE')][(pandas['PURPOSE']==8)] = 4

pandas = pandas[(pandas['AGE'] !=6 ) & \
                #(pandas['INCOME'] !=4 ) &\
                (pandas['PURPOSE'] !=9 ) &\
                (pandas['CHOICE'] !=0 )]


#define dictionaries for visualization  of variables names
age_dic = {1: 'age≤24', 2: '24<age≤39', 3: '39<age≤54', 4: '54<age≤65', 5: '65<age', 6: 'not known'}
purpose_dic = {1: 'Commuter', 2: 'Shopping', 3: 'Business', 4: 'Leisure', 5: 'Return from work', 6: 'Return from shopping', 7: 'Return from business', 8: 'Return from leisure', 9: 'other'}
choice_dic = {1: 'Train', 2: 'SM', 3: 'Car'}
pandas.loc[:, ('INCOME')][(pandas['INCOME']==0)] = 1
income_dic = {1: 'under 50', 2: 'between 50 and 100', 3: 'over 100', 4: 'unknown'}
dic = [age_dic, income_dic, purpose_dic, choice_dic]
ticket_dic = {0: 'None', 1: 'Two way with half price card', 2: 'One way with half price card', 3: 'Two way normal price', 4: 'One way normal price', 5: 'Half day', 6: 'Annual season ticket', 7: 'Annual season ticket Junior or Senior', 8: 'Free travel after 7pm card', 9: 'Group ticket', 10: 'Other'}

#load data into  database
database = db.Database("swissmetro",pandas)
globals().update(database.variables)
database.panel('ID')

## Define the draw generator
myRandomNumberGenerators = {'MLHS':draws.getLatinHypercubeDraws}
database.setRandomNumberGenerators(myRandomNumberGenerators)


#define variables
#variables
CAR_AV_SP =  DefineVariable('CAR_AV_SP',CAR_AV  * (  SP   !=  0  ),database)
TRAIN_AV_SP =  DefineVariable('TRAIN_AV_SP',TRAIN_AV  * (  SP   !=  0  ),database)

SM_COST    =  SM_CO   * (  GA   ==  0  )   
TRAIN_COST =  TRAIN_CO   * (  GA   ==  0  )

###Parameters to be estimated (Note not all parameters are used in all models!)
##Attributes
#Alternative specific constants
ASC_CAR        = Beta('ASC_CAR',0,None,None,1)
ASC_TRAIN      = Beta('ASC_TRAIN',0,None,None,0)
ASC_SM         = Beta('ASC_SM',0,None,None,0)

#Cost (Note: Assumed generic)
B_COST          = Beta('B_COST',0,None,None,0)
B_COST_BUSINESS = Beta('B_COST_BUSINESS',0,None,None,0)
B_COST_PRIVATE  = Beta('B_COST_PRIVATE',0,None,None,0)

#Time
B_TIME                = Beta('B_TIME',0,None,None,0)
B_TIME_CAR            = Beta('B_TIME_CAR',0,None,None,0)
B_TIME_TRAIN          = Beta('B_TIME_TRAIN',0,None,None,0)
B_TIME_SM             = Beta('B_TIME_SM',0,None,None,0)
B_TIME_PUB            = Beta('B_TIME_PUB',0,None,None,0)
B_TIME_CAR_BUSINESS   = Beta('B_TIME_CAR_BUSINESS',0,None,None,0)
B_TIME_TRAIN_BUSINESS = Beta('B_TIME_TRAIN_BUSINESS',0,None,None,0)
B_TIME_SM_BUSINESS    = Beta('B_TIME_SM_BUSINESS',0,None,None,0)
B_TIME_PUB_BUSINESS   = Beta('B_TIME_PUB_BUSINESS',0,None,None,0)
B_TIME_CAR_PRIVATE    = Beta('B_TIME_CAR_PRIVATE',0,None,None,0)
B_TIME_TRAIN_PRIVATE  = Beta('B_TIME_TRAIN_PRIVATE',0,None,None,0)
B_TIME_SM_PRIVATE     = Beta('B_TIME_SM_PRIVATE',0,None,None,0)
B_TIME_PUB_PRIVATE    = Beta('B_TIME_PUB_PRIVATE',0,None,None,0)

#HE (Note: Not available for car)
B_HE                = Beta('B_HE',0,None,None,0)
B_HE_TRAIN          = Beta('B_HE_TRAIN',0,None,None,0)
B_HE_SM             = Beta('B_HE_SM',0,None,None,0)
B_HE_BUSINESS       = Beta('B_HE_BUSINESS',0,None,None,0)
B_HE_TRAIN_BUSINESS = Beta('B_HE_TRAIN_BUSINESS',0,None,None,0)
B_HE_SM_BUSINESS    = Beta('B_HE_SM_BUSINESS',0,None,None,0)
B_HE_PRIVATE        = Beta('B_HE_PRIVATE',0,None,None,0)
B_HE_TRAIN_PRIVATE  = Beta('B_HE_TRAIN_PRIVATE',0,None,None,0)
B_HE_SM_PRIVATE     = Beta('B_HE_SM_PRIVATE',0,None,None,0)

#Seats (Note: Only avaliable for SM)
B_SEATS = Beta('B_SEATS',0,None,None,0)

##Characteristics
#Age
B_AGE_1_TRAIN      = Beta('B_AGE_1_TRAIN',0,None,None,1) #Note: Reference
B_AGE_2_TRAIN      = Beta('B_AGE_2_TRAIN',0,None,None,0)
B_AGE_3_TRAIN      = Beta('B_AGE_3_TRAIN',0,None,None,0)
B_AGE_4_TRAIN      = Beta('B_AGE_4_TRAIN',0,None,None,0)
B_AGE_5_TRAIN      = Beta('B_AGE_5_TRAIN',0,None,None,0)
B_AGE_6_TRAIN      = Beta('B_AGE_6_TRAIN',0,None,None,0)
B_AGE_1_SM         = Beta('B_AGE_1_SM',0,None,None,1) #Note: Reference
B_AGE_2_SM         = Beta('B_AGE_2_SM',0,None,None,0)
B_AGE_3_SM         = Beta('B_AGE_3_SM',0,None,None,0)
B_AGE_4_SM         = Beta('B_AGE_4_SM',0,None,None,0)
B_AGE_5_SM         = Beta('B_AGE_5_SM',0,None,None,0)
B_AGE_6_SM         = Beta('B_AGE_6_SM',0,None,None,0)
B_AGE_1_PUB        = Beta('B_AGE_1_PUB',0,None,None,1) #Note: Reference
B_AGE_2_PUB        = Beta('B_AGE_2_PUB',0,None,None,0)
B_AGE_3_PUB        = Beta('B_AGE_3_PUB',0,None,None,0)
B_AGE_4_PUB        = Beta('B_AGE_4_PUB',0,None,None,0)
B_AGE_5_PUB        = Beta('B_AGE_5_PUB',0,None,None,0)
B_AGE_6_PUB        = Beta('B_AGE_6_PUB',0,None,None,0)
B_AGE_ADULTS_TRAIN = Beta('B_AGE_TRAIN_ADULTS',0,None,None,0)
B_AGE_ADULTS_SM    = Beta('B_AGE_ADULTS_SM',0,None,None,0)
B_AGE_ADULTS_PUB   = Beta('B_AGE_ADULTS_PUB',0,None,None,0)

#Luggage
B_LUGGAGE_TRAIN = Beta('B_LUGGAGE_TRAIN', 0, None, None, 0)
B_LUGGAGE_SM    = Beta('B_LUGGAGE_SM', 0, None, None, 0)
B_LUGGAGE_PUB   = Beta('B_LUGGAGE_PUB', 0, None, None, 0)

#Gender
B_MALE_TRAIN = Beta('B_MALE_TRAIN',0,None,None,0)
B_MALE_SM    = Beta('B_MALE_SM',0,None,None,0)
B_MALE_PUB   = Beta('B_MALE_PUB',0,None,None,0)

#Purpose
B_BUSINESS       = Beta('B_BUSINESS',0,None,None,0)
B_BUSINESS_TRAIN = Beta('B_BUSINESS_TRAIN',0,None,None,0)
B_BUSINESS_SM    = Beta('B_BUSINESS_SM',0,None,None,0)
B_PRIVATE        = Beta('B_PRIVATE',0,None,None,0)
B_PRIVATE_TRAIN  = Beta('B_PRIVATE_TRAIN',0,None,None,0)
B_PRIVATE_SM     = Beta('B_PRIVATE_SM',0,None,None,0)
B_COMMUTER       = Beta('B_COMMUTER',0,None,None,0)
B_COMMUTER_TRAIN = Beta('B_COMMUTER_TRAIN',0,None,None,0)
B_COMMUTER_SM    = Beta('B_COMMUTER_SM',0,None,None,0)

#GA
B_GA        = Beta('B_GA',0,None,None,0)
B_GA_TRAIN  = Beta('B_GA_TRAIN',0,None,None,0)
B_GA_SM     = Beta('B_GA_SM',0,None,None,0)

#First
B_FIRST_TRAIN = Beta('B_FIRST_TRAIN',0,None,None,0)
B_FIRST_SM    = Beta('B_FIRST_SM',0,None,None,0)
B_FIRST       = Beta('B_FIRST',0,None,None,0)

##Non linearization
#Cost
q_COST = Beta('q_COST',1,None,None,0)

#Time
q_TIME       = Beta('q_TIME',1,None,None,0)
q_TIME_TRAIN = Beta('q_TIME_TRAIN',1,None,None,0)
q_TIME_SM    = Beta('q_TIME_SM',1,None,None,0)
q_TIME_CAR   = Beta('q_TIME_CAR',1,None,None,0)
q_TIME_PUB   = Beta('q_TIME_PUB',1,None,None,0)

#HE
q_HE = Beta('q_HE',1,None,None,0)

##Nesting parameter
MU = Beta('MU',1,0,1,0)

##ML RANDOM GENERIC TIME LOGNORMAL
BETA_TIME_mean = Beta('BETA_TIME_mean',0,None,None,0)
BETA_TIME_std = Beta('BETA_TIME_std',1,None,None,0)
BETA_TIME_random = -exp(BETA_TIME_mean + BETA_TIME_std * bioDraws('BETA_TIME_random','NORMAL'))

##ML RANDOM SPECIFIC TIME TRAIN LOGNORMAL
BETA_TIME_TRAIN_mean = Beta('BETA_TIME_TRAIN_mean',0,None,None,0)
BETA_TIME_TRAIN_std = Beta('BETA_TIME_TRAIN_std',1,None,None,0)
BETA_TIME_TRAIN_random = -exp(BETA_TIME_TRAIN_mean + BETA_TIME_TRAIN_std * bioDraws('BETA_TIME_TRAIN_random','NORMAL'))

##ML RANDOM SPECIFIC TIME SM  LOGNORMAL
BETA_TIME_SM_mean = Beta('BETA_TIME_SM_mean',0,None,None,0)
BETA_TIME_SM_std = Beta('BETA_TIME_SM_std',1,None,None,0)
BETA_TIME_SM_random = -exp(BETA_TIME_SM_mean + BETA_TIME_SM_std * bioDraws('BETA_TIME_SM_random','NORMAL'))

##ML RANDOM SPECIFIC TIME CAR LOGNORMAL
BETA_TIME_CAR_mean = Beta('BETA_TIME_CAR_mean',0,None,None,0)
BETA_TIME_CAR_std = Beta('BETA_TIME_CAR_std',1,None,None,0)
BETA_TIME_CAR_random = -exp(BETA_TIME_CAR_mean + BETA_TIME_CAR_std * bioDraws('BETA_TIME_CAR_random','NORMAL'))

##ML RANDOM GENERIC COST LOGNORMAL
BETA_COST_mean = Beta('BETA_COST_mean',0,None,None,0)
BETA_COST_std = Beta('BETA_COST_std',1,None,None,0)
BETA_COST_random = -exp(BETA_COST_mean + BETA_COST_std * bioDraws('BETA_COST_random','NORMAL'))

##ML RANDOM GENERIC HE LOGNORMAL
BETA_HE_mean = Beta('BETA_HE_mean',0,None,None,0)
BETA_HE_std = Beta('BETA_HE_std',1,None,None,0)
BETA_HE_random = -exp(BETA_HE_mean + BETA_HE_std * bioDraws('BETA_HE_random','NORMAL'))

##ML RANDOM GENERIC TIME LOGNORMAL
BBETA_TIME_mean = Beta('BBETA_TIME_mean',0,None,None,0)
BBETA_TIME_std = Beta('BBETA_TIME_std',1,None,None,0)
BBETA_TIME_random = -exp(BBETA_TIME_mean + BBETA_TIME_std * bioDraws('BBETA_TIME_random','NORMAL'))

##ML RANDOM SPECIFIC TIME TRAIN LOGNORMAL
BBETA_TIME_TRAIN_mean = Beta('BBETA_TIME_TRAIN_mean',0,None,None,0)
BBETA_TIME_TRAIN_std = Beta('BBETA_TIME_TRAIN_std',1,None,None,0)
BBETA_TIME_TRAIN_random = -exp(BBETA_TIME_TRAIN_mean + BBETA_TIME_TRAIN_std * bioDraws('BBETA_TIME_TRAIN_random','NORMAL'))

##ML RANDOM SPECIFIC TIME SM  LOGNORMAL
BBETA_TIME_SM_mean = Beta('BBETA_TIME_SM_mean',0,None,None,0)
BBETA_TIME_SM_std = Beta('BBETA_TIME_SM_std',1,None,None,0)
BBETA_TIME_SM_random = -exp(BBETA_TIME_SM_mean + BBETA_TIME_SM_std * bioDraws('BBETA_TIME_SM_random','NORMAL'))

##ML RANDOM SPECIFIC TIME CAR LOGNORMAL
BBETA_TIME_CAR_mean = Beta('BBETA_TIME_CAR_mean',0,None,None,0)
BBETA_TIME_CAR_std = Beta('BBETA_TIME_CAR_std',1,None,None,0)
BBETA_TIME_CAR_random = -exp(BBETA_TIME_CAR_mean + BBETA_TIME_CAR_std * bioDraws('BBETA_TIME_CAR_random','NORMAL'))

##ML RANDOM GENERIC COST LOGNORMAL
BBETA_COST_mean = Beta('BBETA_COST_mean',0,None,None,0)
BBETA_COST_std = Beta('BBETA_COST_std',1,None,None,0)
BBETA_COST_random = -exp(BBETA_COST_mean + BBETA_COST_std * bioDraws('BBETA_COST_random','NORMAL'))

##ML RANDOM GENERIC HE LOGNORMAL
BBETA_HE_mean = Beta('BBETA_HE_mean',0,None,None,0)
BBETA_HE_std = Beta('BBETA_HE_std',1,None,None,0)
BBETA_HE_random = -exp(BBETA_HE_mean + BBETA_HE_std * bioDraws('BBETA_HE_random','NORMAL'))

##ML RANDOM GENERIC TIME NORMAL
BETA_TIME_mean_Norm = Beta('BETA_TIME_mean_Norm',0,None,None,0)
BETA_TIME_std_Norm = Beta('BETA_TIME_std_Norm',1,None,None,0)
BETA_TIME_random_Norm = BETA_TIME_mean_Norm + BETA_TIME_std_Norm * bioDraws('BETA_TIME_random_Norm','NORMAL')

##ML RANDOM SPECIFIC TIME TRAIN LOGNORMAL
BETA_TIME_TRAIN_mean_Norm = Beta('BETA_TIME_TRAIN_mean_Norm',0,None,None,0)
BETA_TIME_TRAIN_std_Norm = Beta('BETA_TIME_TRAIN_std_Norm',1,None,None,0)
BETA_TIME_TRAIN_random_Norm = BETA_TIME_TRAIN_mean_Norm + BETA_TIME_TRAIN_std_Norm * bioDraws('BETA_TIME_TRAIN_random_Norm','NORMAL')

##ML RANDOM SPECIFIC TIME SM  NORMAL
BETA_TIME_SM_mean_Norm = Beta('BETA_TIME_SM_mean_Norm',0,None,None,0)
BETA_TIME_SM_std_Norm = Beta('BETA_TIME_SM_std_Norm',1,None,None,0)
BETA_TIME_SM_random_Norm = BETA_TIME_SM_mean_Norm + BETA_TIME_SM_std_Norm * bioDraws('BETA_TIME_SM_random_Norm','NORMAL')

##ML RANDOM SPECIFIC TIME CAR NORMAL
BETA_TIME_CAR_mean_Norm = Beta('BETA_TIME_CAR_mean_Norm',0,None,None,0)
BETA_TIME_CAR_std_Norm = Beta('BETA_TIME_CAR_std_Norm',1,None,None,0)
BETA_TIME_CAR_random_Norm = BETA_TIME_CAR_mean_Norm + BETA_TIME_CAR_std_Norm * bioDraws('BETA_TIME_CAR_random_Norm','NORMAL')

##ML RANDOM GENERIC COST NORMAL
BETA_COST_mean_Norm = Beta('BETA_COST_mean_Norm',0,None,None,0)
BETA_COST_std_Norm = Beta('BETA_COST_std_Norm',1,None,None,0)
BETA_COST_random_Norm = BETA_COST_mean_Norm + BETA_COST_std_Norm * bioDraws('BETA_COST_random_Norm','NORMAL')

##ML RANDOM GENERIC HE NORMAL
BETA_HE_mean_Norm = Beta('BETA_HE_mean_Norm',0,None,None,0)
BETA_HE_std_Norm = Beta('BETA_HE_std_Norm',1,None,None,0)
BETA_HE_random_Norm = BETA_HE_mean_Norm + BETA_HE_std_Norm * bioDraws('BETA_HE_random_Norm','NORMAL')

##ML RANDOM GENERIC TIME NORMAL
BBETA_TIME_mean_Norm = Beta('BBETA_TIME_mean_Norm',0,None,None,0)
BBETA_TIME_std_Norm = Beta('BBETA_TIME_std_Norm',1,None,None,0)
BBETA_TIME_random_Norm = BBETA_TIME_mean_Norm + BBETA_TIME_std_Norm * bioDraws('BBETA_TIME_random_Norm','NORMAL')

##ML RANDOM SPECIFIC TIME TRAIN LOGNORMAL
BBETA_TIME_TRAIN_mean_Norm = Beta('BBETA_TIME_TRAIN_mean_Norm',0,None,None,0)
BBETA_TIME_TRAIN_std_Norm = Beta('BBETA_TIME_TRAIN_std_Norm',1,None,None,0)
BBETA_TIME_TRAIN_random_Norm = BBETA_TIME_TRAIN_mean_Norm + BBETA_TIME_TRAIN_std_Norm * bioDraws('BBETA_TIME_TRAIN_random_Norm','NORMAL')

##ML RANDOM SPECIFIC TIME SM  NORMAL
BBETA_TIME_SM_mean_Norm = Beta('BBETA_TIME_SM_mean_Norm',0,None,None,0)
BBETA_TIME_SM_std_Norm = Beta('BBETA_TIME_SM_std_Norm',1,None,None,0)
BBETA_TIME_SM_random_Norm = BBETA_TIME_SM_mean_Norm + BBETA_TIME_SM_std_Norm * bioDraws('BBETA_TIME_SM_random_Norm','NORMAL')

##ML RANDOM SPECIFIC TIME CAR NORMAL
BBETA_TIME_CAR_mean_Norm = Beta('BBETA_TIME_CAR_mean_Norm',0,None,None,0)
BBETA_TIME_CAR_std_Norm = Beta('BBETA_TIME_CAR_std_Norm',1,None,None,0)
BBETA_TIME_CAR_random_Norm = BBETA_TIME_CAR_mean_Norm + BBETA_TIME_CAR_std_Norm * bioDraws('BBETA_TIME_CAR_random_Norm','NORMAL')

##ML RANDOM GENERIC COST NORMAL
BBETA_COST_mean_Norm = Beta('BBETA_COST_mean_Norm',0,None,None,0)
BBETA_COST_std_Norm = Beta('BBETA_COST_std_Norm',1,None,None,0)
BBETA_COST_random_Norm = BBETA_COST_mean_Norm + BBETA_COST_std_Norm * bioDraws('BBETA_COST_random_Norm','NORMAL')

##ML RANDOM GENERIC HE NORMAL
BBETA_HE_mean_Norm = Beta('BBETA_HE_mean_Norm',0,None,None,0)
BBETA_HE_std_Norm = Beta('BBETA_HE_std_Norm',1,None,None,0)
BBETA_HE_random_Norm = BBETA_HE_mean_Norm + BBETA_HE_std_Norm * bioDraws('BBETA_HE_random_Norm','NORMAL')

##Scaling 'COST', 'TRAVEL-TIME' and 'HE' by a factor of 100 and adding the scaled variables to the database
 
TRAIN_TT_SCALED = DefineVariable('TRAIN_TT_SCALED',\
                                 TRAIN_TT / 100.0,database)
TRAIN_COST_SCALED = DefineVariable('TRAIN_COST_SCALED',\
                                   TRAIN_COST / 100,database)
TRAIN_HE_SCALED = DefineVariable('TRAIN_HE_SCALED',\
                                   TRAIN_HE / 100, database)

SM_TT_SCALED = DefineVariable('SM_TT_SCALED', SM_TT / 100.0,database)
SM_COST_SCALED = DefineVariable('SM_COST_SCALED', SM_COST / 100,database)
SM_HE_SCALED = DefineVariable('SM_HE_SCALED',\
                                   SM_HE / 100, database)

CAR_TT_SCALED = DefineVariable('CAR_TT_SCALED', CAR_TT / 100,database)
CAR_COST_SCALED = DefineVariable('CAR_COST_SCALED', CAR_CO / 100,database)

###Defining new variables and adding columns to the database
#Age
AGE_1 = DefineVariable('AGE_1', (  AGE   ==  1  ),database) #don't scale because is cathegorical
AGE_2 = DefineVariable('AGE_2', (  AGE   ==  2  ),database)
AGE_3 = DefineVariable('AGE_3', (  AGE   ==  3  ),database)
AGE_4 = DefineVariable('AGE_4', (  AGE   ==  4  ),database)
AGE_5 = DefineVariable('AGE_5', (  AGE   ==  5  ),database)
AGE_6 = DefineVariable('AGE_6', (  AGE   ==  6  ),database)

#Purpose
PRIVATE  = DefineVariable("PRIVATE", (PURPOSE == 1), database)
COMMUTER = DefineVariable("COMMUTER", (PURPOSE == 2), database)
BUSINESS = DefineVariable("BUSINESS", (PURPOSE == 3), database)

In [6]:
vSeq = []
biogemeSeq = []
bestResultPosition = []
ModNameSeqence = []
best = []
results = []
Draws=100

In [7]:
#BETA_COST_random

#Utility functions
TRAIN_SP = ASC_TRAIN                                      + \
     BETA_COST_random             * TRAIN_COST_SCALED + \
     B_TIME_TRAIN       * TRAIN_TT_SCALED   + \
     B_HE               * TRAIN_HE_SCALED           + \
     B_AGE_ADULTS_TRAIN * AGE_2                     + \
     B_AGE_3_TRAIN      * AGE_3                     + \
     B_AGE_ADULTS_TRAIN * AGE_4                     + \
     B_AGE_5_TRAIN      * AGE_5                     + \
     B_LUGGAGE_TRAIN    * LUGGAGE                   + \
     B_MALE_TRAIN       * MALE                      + \
     B_GA_TRAIN         * GA                        + \
     B_BUSINESS_TRAIN   * BUSINESS                  + \
     B_PRIVATE          * PRIVATE                   + \
     B_COMMUTER         * COMMUTER

SM_SP = ASC_SM                                   + \
     BETA_COST_random          * SM_COST_SCALED + \
     B_TIME_SM       * SM_TT_SCALED   + \
     B_HE            * SM_HE_SCALED           + \
     B_AGE_ADULTS_SM * AGE_2                  + \
     B_AGE_ADULTS_SM * AGE_3                  + \
     B_AGE_4_SM      * AGE_4                  + \
     B_AGE_5_SM      * AGE_5                  + \
     B_FIRST_SM      * FIRST                  + \
     B_BUSINESS_SM   * BUSINESS               + \
     B_PRIVATE       * PRIVATE                + \
     B_COMMUTER      * COMMUTER

Car_SP = ASC_CAR                              + \
     BETA_COST_random     * CAR_COST_SCALED + \
     B_TIME_CAR * CAR_TT_SCALED



V = {3: Car_SP,1: TRAIN_SP,2: SM_SP}
av = {3: CAR_AV_SP,1: TRAIN_AV_SP,2: SM_AV}

vSeq.append(V)
#logProbSeq.append(logprob)
ModNameSeqence.append("ML_"+str(len(vSeq)))
display(Markdown(f"# ^^^^^^^^^^^"))
display(Markdown(f"# {ModNameSeqence[-1]}"))

# ^^^^^^^^^^^

# ML_1

In [8]:
# database.panel('ID')
# Conditional to B_TIME_RND, the likelihood of one observation is
# given by the logit model (called the kernel)
obsprob = exp(models.loglogit(V,av,CHOICE))

# Conditional to B_TIME_RND, the likelihood of all observations for
# one individual (the trajectory) is the product of the likelihood of
# each observation.
condprobIndiv = PanelLikelihoodTrajectory(obsprob)

# We integrate over B_TIME_RND using Monte-Carlo
logprob = log(MonteCarlo(condprobIndiv))

# Create the Biogeme object
biogeme = bio.BIOGEME(database,logprob,numberOfDraws=Draws)
biogeme.modelName = ModNameSeqence[-1]

# Estimate the parameters
results.append(biogeme.estimate())

In [9]:
print_result(results, database)

# Results model ML_1

                       Value        Std err         t-test  p-value
ASC_SM              0.129027  1.797693e+308  7.177359e-310      1.0
ASC_TRAIN          -0.667315  1.797693e+308 -3.712062e-309      1.0
BETA_COST_mean      0.498236  1.797693e+308  2.771532e-309      1.0
BETA_COST_std       1.976348  1.797693e+308  1.099380e-308      1.0
B_AGE_3_TRAIN      -1.379531   0.000000e+00  1.797693e+308      0.0
B_AGE_4_SM         -0.886321  5.273843e-155 -1.680598e+154      0.0
B_AGE_5_SM         -1.064814  -0.000000e+00  1.797693e+308      0.0
B_AGE_5_TRAIN      -0.068540   0.000000e+00  1.797693e+308      0.0
B_AGE_ADULTS_SM    -0.607723  2.222759e-162 -2.734094e+161      0.0
B_AGE_TRAIN_ADULTS -1.354971  5.273843e-155 -2.569229e+154      0.0
B_BUSINESS_SM       0.756189   0.000000e+00  1.797693e+308      0.0
B_BUSINESS_TRAIN    1.242377   0.000000e+00  1.797693e+308      0.0
B_COMMUTER          1.864058  5.273843e-155  3.534534e+154      0.0
B_FIRST_SM          0.250323  1.797693e+308  1.3

In [10]:
#BETA_COST_random_Norm



#Utility functions
TRAIN_SP = ASC_TRAIN                                      + \
     BETA_COST_random_Norm             * TRAIN_COST_SCALED + \
     B_TIME_TRAIN       * TRAIN_TT_SCALED   + \
     B_HE               * TRAIN_HE_SCALED           + \
     B_AGE_ADULTS_TRAIN * AGE_2                     + \
     B_AGE_3_TRAIN      * AGE_3                     + \
     B_AGE_ADULTS_TRAIN * AGE_4                     + \
     B_AGE_5_TRAIN      * AGE_5                     + \
     B_LUGGAGE_TRAIN    * LUGGAGE                   + \
     B_MALE_TRAIN       * MALE                      + \
     B_GA_TRAIN         * GA                        + \
     B_BUSINESS_TRAIN   * BUSINESS                  + \
     B_PRIVATE          * PRIVATE                   + \
     B_COMMUTER         * COMMUTER

SM_SP = ASC_SM                                   + \
     BETA_COST_random_Norm * SM_COST_SCALED + \
     B_TIME_SM       * SM_TT_SCALED   + \
     B_HE            * SM_HE_SCALED           + \
     B_AGE_ADULTS_SM * AGE_2                  + \
     B_AGE_ADULTS_SM * AGE_3                  + \
     B_AGE_4_SM      * AGE_4                  + \
     B_AGE_5_SM      * AGE_5                  + \
     B_FIRST_SM      * FIRST                  + \
     B_BUSINESS_SM   * BUSINESS               + \
     B_PRIVATE       * PRIVATE                + \
     B_COMMUTER      * COMMUTER

Car_SP = ASC_CAR                              + \
     BETA_COST_random_Norm     * CAR_COST_SCALED + \
     B_TIME_CAR * CAR_TT_SCALED



V = {3: Car_SP,1: TRAIN_SP,2: SM_SP}
av = {3: CAR_AV_SP,1: TRAIN_AV_SP,2: SM_AV}

vSeq.append(V)
ModNameSeqence.append("ML_"+str(len(vSeq)))
display(Markdown(f"# ^^^^^^^^^^^"))
display(Markdown(f"# {ModNameSeqence[-1]}"))

# ^^^^^^^^^^^

# ML_2

In [11]:
bestRes, results = estimate_N_compare(ModNameSeqence[-1]
                               , 'ML_1'
                               , vSeq
                               , av
                               , CHOICE, Car_SP, TRAIN_SP, SM_SP, CAR_AV_SP, TRAIN_AV_SP, SM_AV
                               , database
                               , pandas
                               , myRandomNumberGenerators
                               , results
                               , ModNameSeqence=ModNameSeqence
                               , Draws=Draws)
best.append(bestRes)

## Estimate  ML_2 and compare with ML_1

#### Unrestricted model ML_2 and Restricted model ML_1

# Results model ML_2

                        Value   Std err     t-test       p-value
ASC_SM              -0.048531  0.306759  -0.158204  8.742960e-01
ASC_TRAIN           -0.453333  0.334291  -1.356103  1.750665e-01
BETA_COST_mean_Norm -3.434380  0.171492 -20.026479  0.000000e+00
BETA_COST_std_Norm   4.282128  0.198421  21.581015  0.000000e+00
B_AGE_3_TRAIN       -1.446603  0.292266  -4.949607  7.436345e-07
B_AGE_4_SM          -1.111647  0.288948  -3.847223  1.194644e-04
B_AGE_5_SM          -1.432079  0.313436  -4.568971  4.901251e-06
B_AGE_5_TRAIN       -0.294204  0.318542  -0.923593  3.556983e-01
B_AGE_ADULTS_SM     -0.765477  0.282352  -2.711072  6.706604e-03
B_AGE_TRAIN_ADULTS  -1.429918  0.287841  -4.967731  6.774099e-07
B_BUSINESS_SM        0.812856  0.092137   8.822239  0.000000e+00
B_BUSINESS_TRAIN     1.295611  0.109837  11.795750  0.000000e+00
B_COMMUTER           1.555806  0.170214   9.140314  0.000000e+00
B_FIRST_SM           0.367211  0.064215   5.718466  1.074902e-08
B_GA_TRAIN           1.81

# Results model ML_1

...

#### LogLikelihood TEST None disagrees with BenAkivaSwaitHorowitzTest ML_1

### TRY  CROSS VALIDATION

# Perform  Cross Validation Between ['ML_2', 'ML_1']

Model 0
Fold 0
Model 0
Fold 1
Model 0
Fold 2
Model 0
Fold 3
Model 0
Fold 4
Model 1
Fold 0
Model 1
Fold 1
Model 1
Fold 2
Model 1
Fold 3
Model 1
Fold 4
loglikelihood model ML_1: -1532.7459204911288 +/- 34.890709068029295
loglikelihood model ML_2: -1548.8919477587963 +/- 52.269912673525155


## Accept ML_1 performs better off the sample

In [12]:
#BETA_COST_random
#BETA_HE_random




#Utility functions
TRAIN_SP = ASC_TRAIN                                      + \
     BETA_COST_random             * TRAIN_COST_SCALED + \
     B_TIME_TRAIN       * TRAIN_TT_SCALED   + \
     BETA_HE_random               * TRAIN_HE_SCALED           + \
     B_AGE_ADULTS_TRAIN * AGE_2                     + \
     B_AGE_3_TRAIN      * AGE_3                     + \
     B_AGE_ADULTS_TRAIN * AGE_4                     + \
     B_AGE_5_TRAIN      * AGE_5                     + \
     B_LUGGAGE_TRAIN    * LUGGAGE                   + \
     B_MALE_TRAIN       * MALE                      + \
     B_GA_TRAIN         * GA                        + \
     B_BUSINESS_TRAIN   * BUSINESS                  + \
     B_PRIVATE          * PRIVATE                   + \
     B_COMMUTER         * COMMUTER

SM_SP = ASC_SM                                   + \
     BETA_COST_random * SM_COST_SCALED + \
     B_TIME_SM       * SM_TT_SCALED   + \
     BETA_HE_random            * SM_HE_SCALED           + \
     B_AGE_ADULTS_SM * AGE_2                  + \
     B_AGE_ADULTS_SM * AGE_3                  + \
     B_AGE_4_SM      * AGE_4                  + \
     B_AGE_5_SM      * AGE_5                  + \
     B_FIRST_SM      * FIRST                  + \
     B_BUSINESS_SM   * BUSINESS               + \
     B_PRIVATE       * PRIVATE                + \
     B_COMMUTER      * COMMUTER

Car_SP = ASC_CAR                              + \
     BETA_COST_random     * CAR_COST_SCALED + \
     B_TIME_CAR * CAR_TT_SCALED



V = {3: Car_SP,1: TRAIN_SP,2: SM_SP}
av = {3: CAR_AV_SP,1: TRAIN_AV_SP,2: SM_AV}

vSeq.append(V)
ModNameSeqence.append("ML_"+str(len(vSeq)))
display(Markdown(f"# ^^^^^^^^^^^"))
display(Markdown(f"# {ModNameSeqence[-1]}"))

# ^^^^^^^^^^^

# ML_3

In [13]:
bestRes, results = estimate_N_compare(ModNameSeqence[-1]
                               , bestRes
                               , vSeq
                               , av
                               , CHOICE, Car_SP, TRAIN_SP, SM_SP, CAR_AV_SP, TRAIN_AV_SP, SM_AV
                               , database
                               , pandas
                               , myRandomNumberGenerators
                               , results
                               , ModNameSeqence=ModNameSeqence
                               , Draws=Draws)
best.append(bestRes)

## Estimate  ML_3 and compare with ML_1

#### Unrestricted model ML_3 and Restricted model ML_1

# Results model ML_3

                       Value        Std err         t-test  p-value
ASC_SM             -0.028358  1.797693e+308 -1.577443e-310      1.0
ASC_TRAIN          -0.463904  1.797693e+308 -2.580550e-309      1.0
BETA_COST_mean      0.811610  1.797693e+308  4.514731e-309      1.0
BETA_COST_std       1.864530  1.797693e+308  1.037179e-308      1.0
BETA_HE_mean        0.471902  1.797693e+308  2.625039e-309      1.0
BETA_HE_std         1.848860  1.797693e+308  1.028463e-308      1.0
B_AGE_3_TRAIN      -1.584494  3.457293e-155 -4.583049e+154      0.0
B_AGE_4_SM         -0.855949  1.797693e+308 -4.761373e-309      1.0
B_AGE_5_SM         -0.824577  1.728647e-155 -4.770073e+154      0.0
B_AGE_5_TRAIN       0.160130  1.728647e-155  9.263303e+153      0.0
B_AGE_ADULTS_SM    -0.718632  1.797693e+308 -3.997522e-309      1.0
B_AGE_TRAIN_ADULTS -1.486785  -0.000000e+00  1.797693e+308      0.0
B_BUSINESS_SM       0.970819  2.636922e-155  3.681638e+154      0.0
B_BUSINESS_TRAIN    1.355407  2.636922e-155  5.1

# Results model ML_1

...

#### LogLikelihood TEST ML_3 disagrees with BenAkivaSwaitHorowitzTest ML_1

### TRY  CROSS VALIDATION

# Perform  Cross Validation Between ['ML_3', 'ML_1']

Model 0
Fold 0
Model 0
Fold 1
Model 0
Fold 2
Model 0
Fold 3
Model 0
Fold 4
Model 1
Fold 0
Model 1
Fold 1
Model 1
Fold 2
Model 1
Fold 3
Model 1
Fold 4
loglikelihood model ML_1: -1534.3240046912283 +/- 35.531195497015
loglikelihood model ML_3: -1553.5591658670771 +/- 24.911283567154456


## Accept ML_1 performs better off the sample

In [14]:
#BETA_COST_random
#BETA_HE_random_Norm


#Utility functions
TRAIN_SP = ASC_TRAIN                                      + \
     BETA_COST_random             * TRAIN_COST_SCALED + \
     B_TIME_TRAIN       * TRAIN_TT_SCALED   + \
     BETA_HE_random_Norm               * TRAIN_HE_SCALED           + \
     B_AGE_ADULTS_TRAIN * AGE_2                     + \
     B_AGE_3_TRAIN      * AGE_3                     + \
     B_AGE_ADULTS_TRAIN * AGE_4                     + \
     B_AGE_5_TRAIN      * AGE_5                     + \
     B_LUGGAGE_TRAIN    * LUGGAGE                   + \
     B_MALE_TRAIN       * MALE                      + \
     B_GA_TRAIN         * GA                        + \
     B_BUSINESS_TRAIN   * BUSINESS                  + \
     B_PRIVATE          * PRIVATE                   + \
     B_COMMUTER         * COMMUTER

SM_SP = ASC_SM                                   + \
     BETA_COST_random * SM_COST_SCALED + \
     B_TIME_SM       * SM_TT_SCALED   + \
     BETA_HE_random_Norm            * SM_HE_SCALED           + \
     B_AGE_ADULTS_SM * AGE_2                  + \
     B_AGE_ADULTS_SM * AGE_3                  + \
     B_AGE_4_SM      * AGE_4                  + \
     B_AGE_5_SM      * AGE_5                  + \
     B_FIRST_SM      * FIRST                  + \
     B_BUSINESS_SM   * BUSINESS               + \
     B_PRIVATE       * PRIVATE                + \
     B_COMMUTER      * COMMUTER

Car_SP = ASC_CAR                              + \
     BETA_COST_random     * CAR_COST_SCALED + \
     B_TIME_CAR * CAR_TT_SCALED



V = {3: Car_SP,1: TRAIN_SP,2: SM_SP}
av = {3: CAR_AV_SP,1: TRAIN_AV_SP,2: SM_AV}

vSeq.append(V)
ModNameSeqence.append("ML_"+str(len(vSeq)))
display(Markdown(f"# ^^^^^^^^^^^"))
display(Markdown(f"# {ModNameSeqence[-1]}"))

# ^^^^^^^^^^^

# ML_4

In [15]:
bestRes, results = estimate_N_compare(ModNameSeqence[-1]
                               , bestRes
                               , vSeq
                               , av
                               , CHOICE, Car_SP, TRAIN_SP, SM_SP, CAR_AV_SP, TRAIN_AV_SP, SM_AV
                               , database
                               , pandas
                               , myRandomNumberGenerators
                               , results
                               , ModNameSeqence=ModNameSeqence
                               , Draws=Draws)
best.append(bestRes)

## Estimate  ML_4 and compare with ML_1

#### Unrestricted model ML_4 and Restricted model ML_1

# Results model ML_4

                       Value        Std err         t-test  p-value
ASC_SM              0.221515  1.797693e+308  1.232216e-309      1.0
ASC_TRAIN          -0.153759  1.797693e+308 -8.553102e-310      1.0
BETA_COST_mean      0.634772  1.797693e+308  3.531034e-309      1.0
BETA_COST_std       1.931400  1.797693e+308  1.074377e-308      1.0
BETA_HE_mean_Norm  -3.793205  1.797693e+308 -2.110040e-308      1.0
BETA_HE_std_Norm    4.432274  1.797693e+308  2.465534e-308      1.0
B_AGE_3_TRAIN      -1.726406  1.797693e+308 -9.603450e-309      1.0
B_AGE_4_SM         -0.984551  1.797693e+308 -5.476745e-309      1.0
B_AGE_5_SM         -1.104449  1.797693e+308 -6.143699e-309      1.0
B_AGE_5_TRAIN      -0.275204  1.797693e+308 -1.530875e-309      1.0
B_AGE_ADULTS_SM    -0.729667  1.797693e+308 -4.058907e-309      1.0
B_AGE_TRAIN_ADULTS -1.567598  1.797693e+308 -8.720051e-309      1.0
B_BUSINESS_SM       0.951517   0.000000e+00  1.797693e+308      0.0
B_BUSINESS_TRAIN    1.284649   0.000000e+00  1.7

# Results model ML_1

...

### Diff. rho bar square ML_1 > ML_4 <- False

***************
RatioTest 872.9960902785533
Critical 3.841458820694124
Chose Unrestricted (reject H0 the parameters of the unrestricted model are 0)


### LL Ratio Test Result (abs(STAT)-CHI2)869.1546314578592

Probability that the unrestricted model is false having greater rho bar squere, is below threshold:
Threshold:0.07
Probability:9.891948461887619e-61
Difference:-0.07


#### LogLikelihood ML_4 - BenAkivaSwaitHorowitzTest ML_4

In [16]:
####BETA_COST_random
####BETA_HE
####BETA_TIME_TRAIN_random





#Utility functions
TRAIN_SP = ASC_TRAIN                                      + \
     BETA_COST_random             * TRAIN_COST_SCALED + \
     BETA_TIME_TRAIN_random       * TRAIN_TT_SCALED   + \
     B_HE               * TRAIN_HE_SCALED           + \
     B_AGE_ADULTS_TRAIN * AGE_2                     + \
     B_AGE_3_TRAIN      * AGE_3                     + \
     B_AGE_ADULTS_TRAIN * AGE_4                     + \
     B_AGE_5_TRAIN      * AGE_5                     + \
     B_LUGGAGE_TRAIN    * LUGGAGE                   + \
     B_MALE_TRAIN       * MALE                      + \
     B_GA_TRAIN         * GA                        + \
     B_BUSINESS_TRAIN   * BUSINESS                  + \
     B_PRIVATE          * PRIVATE                   + \
     B_COMMUTER         * COMMUTER

SM_SP = ASC_SM                                   + \
     BETA_COST_random * SM_COST_SCALED + \
     B_TIME_SM       * SM_TT_SCALED   + \
     B_HE            * SM_HE_SCALED           + \
     B_AGE_ADULTS_SM * AGE_2                  + \
     B_AGE_ADULTS_SM * AGE_3                  + \
     B_AGE_4_SM      * AGE_4                  + \
     B_AGE_5_SM      * AGE_5                  + \
     B_FIRST_SM      * FIRST                  + \
     B_BUSINESS_SM   * BUSINESS               + \
     B_PRIVATE       * PRIVATE                + \
     B_COMMUTER      * COMMUTER

Car_SP = ASC_CAR                              + \
     BETA_COST_random     * CAR_COST_SCALED + \
     B_TIME_CAR * CAR_TT_SCALED



V = {3: Car_SP,1: TRAIN_SP,2: SM_SP}
av = {3: CAR_AV_SP,1: TRAIN_AV_SP,2: SM_AV}

vSeq.append(V)
ModNameSeqence.append("ML_"+str(len(vSeq)))
display(Markdown(f"# ^^^^^^^^^^^"))
display(Markdown(f"# {ModNameSeqence[-1]}"))

# ^^^^^^^^^^^

# ML_5

In [17]:
bestRes, results = estimate_N_compare(ModNameSeqence[-1]
                               , bestRes
                               , vSeq
                               , av
                               , CHOICE, Car_SP, TRAIN_SP, SM_SP, CAR_AV_SP, TRAIN_AV_SP, SM_AV
                               , database
                               , pandas
                               , myRandomNumberGenerators
                               , results
                               , ModNameSeqence=ModNameSeqence
                               , Draws=Draws)
best.append(bestRes)

## Estimate  ML_5 and compare with ML_4

#### Unrestricted model ML_5 and Restricted model ML_4

# Results model ML_5

                         Value        Std err         t-test  p-value
ASC_SM                0.194887  6.215284e-156  3.135612e+154      0.0
ASC_TRAIN             1.268078  6.215284e-156  2.040258e+155      0.0
BETA_COST_mean        0.589990  6.215284e-156  9.492566e+154      0.0
BETA_COST_std         1.703059  6.215284e-156  2.740115e+155      0.0
BETA_TIME_TRAIN_mean  1.514548  6.215284e-156  2.436812e+155      0.0
BETA_TIME_TRAIN_std   0.648419  6.215284e-156  1.043265e+155      0.0
B_AGE_3_TRAIN        -1.219231  6.089710e-155 -2.002118e+154      0.0
B_AGE_4_SM           -1.053642  6.089710e-155 -1.730200e+154      0.0
B_AGE_5_SM           -1.126488  2.029903e-155 -5.549467e+154      0.0
B_AGE_5_TRAIN         1.875856  2.029903e-155  9.241112e+154      0.0
B_AGE_ADULTS_SM      -0.749966   0.000000e+00  1.797693e+308      0.0
B_AGE_TRAIN_ADULTS   -1.009791  6.089710e-155 -1.658192e+154      0.0
B_BUSINESS_SM         0.709122  3.044855e-155  2.328920e+154      0.0
B_BUSINESS_TRAIN    

# Results model ML_4

...

#### LogLikelihood TEST None disagrees with BenAkivaSwaitHorowitzTest ML_4

### TRY  CROSS VALIDATION

# Perform  Cross Validation Between ['ML_5', 'ML_4']

Model 0
Fold 0
Model 0
Fold 1
Model 0
Fold 2
Model 0
Fold 3
Model 0
Fold 4
Model 1
Fold 0
Model 1
Fold 1
Model 1
Fold 2
Model 1
Fold 3
Model 1
Fold 4
loglikelihood model ML_5: -1548.267368447137 +/- 35.36314783204201
loglikelihood model ML_4: -1568.7698936018212 +/- 29.76161344557146


## Accept ML_5 performs better off the sample

In [18]:
##q_COST
##q_TIME
##q_HE

####BETA_COST_random
####BETA_HE_random_Normal
####BETA_TIME_TRAIN_random_Norm

#BETA_TIME_SM_random
#BETA_TIME_CAR_random





#Utility functions
TRAIN_SP = ASC_TRAIN                                      + \
     BETA_COST_random             * TRAIN_COST_SCALED + \
     BETA_TIME_TRAIN_random_Norm       * TRAIN_TT_SCALED   + \
     BETA_HE_random_Norm               * TRAIN_HE_SCALED           + \
     B_AGE_ADULTS_TRAIN * AGE_2                     + \
     B_AGE_3_TRAIN      * AGE_3                     + \
     B_AGE_ADULTS_TRAIN * AGE_4                     + \
     B_AGE_5_TRAIN      * AGE_5                     + \
     B_LUGGAGE_TRAIN    * LUGGAGE                   + \
     B_MALE_TRAIN       * MALE                      + \
     B_GA_TRAIN         * GA                        + \
     B_BUSINESS_TRAIN   * BUSINESS                  + \
     B_PRIVATE          * PRIVATE                   + \
     B_COMMUTER         * COMMUTER

SM_SP = ASC_SM                                   + \
     BETA_COST_random * SM_COST_SCALED + \
     B_TIME_SM       * SM_TT_SCALED   + \
     BETA_HE_random_Norm            * SM_HE_SCALED           + \
     B_AGE_ADULTS_SM * AGE_2                  + \
     B_AGE_ADULTS_SM * AGE_3                  + \
     B_AGE_4_SM      * AGE_4                  + \
     B_AGE_5_SM      * AGE_5                  + \
     B_FIRST_SM      * FIRST                  + \
     B_BUSINESS_SM   * BUSINESS               + \
     B_PRIVATE       * PRIVATE                + \
     B_COMMUTER      * COMMUTER

Car_SP = ASC_CAR                              + \
     BETA_COST_random     * CAR_COST_SCALED + \
     B_TIME_CAR * CAR_TT_SCALED



V = {3: Car_SP,1: TRAIN_SP,2: SM_SP}
av = {3: CAR_AV_SP,1: TRAIN_AV_SP,2: SM_AV}

vSeq.append(V)
ModNameSeqence.append("ML_"+str(len(vSeq)))
display(Markdown(f"# ^^^^^^^^^^^"))
display(Markdown(f"# {ModNameSeqence[-1]}"))

# ^^^^^^^^^^^

# ML_6

In [19]:
bestRes, results = estimate_N_compare(ModNameSeqence[-1]
                               , bestRes
                               , vSeq
                               , av
                               , CHOICE, Car_SP, TRAIN_SP, SM_SP, CAR_AV_SP, TRAIN_AV_SP, SM_AV
                               , database
                               , pandas
                               , myRandomNumberGenerators
                               , results
                               , ModNameSeqence=ModNameSeqence
                               , Draws=Draws)
best.append(bestRes)

## Estimate  ML_6 and compare with ML_5

#### Unrestricted model ML_6 and Restricted model ML_5

# Results model ML_6

                              Value        Std err         t-test  p-value
ASC_SM                     0.057130  7.458341e-156  7.659843e+153      0.0
ASC_TRAIN                  1.487481  7.458341e-156  1.994386e+155      0.0
BETA_COST_mean             0.706370  7.458341e-156  9.470874e+154      0.0
BETA_COST_std              1.551690  7.458341e-156  2.080476e+155      0.0
BETA_HE_mean_Norm         -2.988320  7.458341e-156 -4.006682e+155      0.0
BETA_HE_std_Norm           4.574239  7.458341e-156  6.133052e+155      0.0
BETA_TIME_TRAIN_mean_Norm -4.080252  7.458341e-156 -5.470724e+155      0.0
BETA_TIME_TRAIN_std_Norm   2.091175  7.458341e-156  2.803808e+155      0.0
B_AGE_3_TRAIN             -1.841660  -0.000000e+00  1.797693e+308      0.0
B_AGE_4_SM                -1.000376  1.797693e+308 -5.564777e-309      1.0
B_AGE_5_SM                -1.146939   0.000000e+00  1.797693e+308      0.0
B_AGE_5_TRAIN              0.442283  -0.000000e+00  1.797693e+308      0.0
B_AGE_ADULTS_SM          

# Results model ML_5

...

### Diff. rho bar square ML_5 > ML_6 <- False

***************
RatioTest 310.63270530054615
Critical 3.841458820694124
Chose Unrestricted (reject H0 the parameters of the unrestricted model are 0)


### LL Ratio Test Result (abs(STAT)-CHI2)306.791246479852

Probability that the unrestricted model is false having greater rho bar squere, is below threshold:
Threshold:0.07
Probability:0.0
Difference:-0.07


#### LogLikelihood ML_6 - BenAkivaSwaitHorowitzTest ML_6

In [20]:
####BETA_COST_random
####BETA_HE_random_Normal
####BETA_TIME_TRAIN_random_Norm
####BETA_TIME_SM_random





#Utility functions
TRAIN_SP = ASC_TRAIN                                      + \
     BETA_COST_random             * TRAIN_COST_SCALED + \
     BETA_TIME_TRAIN_random_Norm       * TRAIN_TT_SCALED   + \
     BETA_HE_random_Norm               * TRAIN_HE_SCALED           + \
     B_AGE_ADULTS_TRAIN * AGE_2                     + \
     B_AGE_3_TRAIN      * AGE_3                     + \
     B_AGE_ADULTS_TRAIN * AGE_4                     + \
     B_AGE_5_TRAIN      * AGE_5                     + \
     B_LUGGAGE_TRAIN    * LUGGAGE                   + \
     B_MALE_TRAIN       * MALE                      + \
     B_GA_TRAIN         * GA                        + \
     B_BUSINESS_TRAIN   * BUSINESS                  + \
     B_PRIVATE          * PRIVATE                   + \
     B_COMMUTER         * COMMUTER

SM_SP = ASC_SM                                   + \
     BETA_COST_random * SM_COST_SCALED + \
     BETA_TIME_SM_random       * SM_TT_SCALED   + \
     BETA_HE_random_Norm       * SM_HE_SCALED   + \
     B_AGE_ADULTS_SM * AGE_2                  + \
     B_AGE_ADULTS_SM * AGE_3                  + \
     B_AGE_4_SM      * AGE_4                  + \
     B_AGE_5_SM      * AGE_5                  + \
     B_FIRST_SM      * FIRST                  + \
     B_BUSINESS_SM   * BUSINESS               + \
     B_PRIVATE       * PRIVATE                + \
     B_COMMUTER      * COMMUTER

Car_SP = ASC_CAR                              + \
     BETA_COST_random     * CAR_COST_SCALED + \
     B_TIME_CAR * CAR_TT_SCALED



V = {3: Car_SP,1: TRAIN_SP,2: SM_SP}
av = {3: CAR_AV_SP,1: TRAIN_AV_SP,2: SM_AV}

vSeq.append(V)
ModNameSeqence.append("ML_"+str(len(vSeq)))
display(Markdown(f"# ^^^^^^^^^^^"))
display(Markdown(f"# {ModNameSeqence[-1]}"))

# ^^^^^^^^^^^

# ML_7

In [21]:
bestRes, results = estimate_N_compare(ModNameSeqence[-1]
                               , bestRes
                               , vSeq
                               , av
                               , CHOICE, Car_SP, TRAIN_SP, SM_SP, CAR_AV_SP, TRAIN_AV_SP, SM_AV
                               , database
                               , pandas
                               , myRandomNumberGenerators
                               , results
                               , ModNameSeqence=ModNameSeqence
                               , Draws=Draws)
best.append(bestRes)

## Estimate  ML_7 and compare with ML_6

#### Unrestricted model ML_7 and Restricted model ML_6

# Results model ML_7

                              Value        Std err         t-test  p-value
ASC_SM                     0.330809  6.524689e-156  5.070110e+154      0.0
ASC_TRAIN                  0.135607  6.524689e-156  2.078361e+154      0.0
BETA_COST_mean             1.034880  6.524689e-156  1.586099e+155      0.0
BETA_COST_std              1.468602  6.524689e-156  2.250839e+155      0.0
BETA_HE_mean_Norm         -1.506154  6.524689e-156 -2.308392e+155      0.0
BETA_HE_std_Norm           1.353048  6.524689e-156  2.073735e+155      0.0
BETA_TIME_SM_mean          1.383183  6.524689e-156  2.119921e+155      0.0
BETA_TIME_SM_std           0.643644  6.524689e-156  9.864739e+154      0.0
BETA_TIME_TRAIN_mean_Norm -5.284613  6.524689e-156 -8.099411e+155      0.0
BETA_TIME_TRAIN_std_Norm   1.840412  6.524689e-156  2.820689e+155      0.0
B_AGE_3_TRAIN             -1.520695  -0.000000e+00  1.797693e+308      0.0
B_AGE_4_SM                -1.169901   0.000000e+00  1.797693e+308      0.0
B_AGE_5_SM               

# Results model ML_6

...

### Diff. rho bar square ML_6 > ML_7 <- False

***************
RatioTest 877.0894567017476
Critical 3.841458820694124
Chose Unrestricted (reject H0 the parameters of the unrestricted model are 0)


### LL Ratio Test Result (abs(STAT)-CHI2)873.2479978810535

Probability that the unrestricted model is false having greater rho bar squere, is below threshold:
Threshold:0.07
Probability:0.0
Difference:-0.07


#### LogLikelihood ML_7 - BenAkivaSwaitHorowitzTest ML_7

In [22]:
####BETA_COST_random
####BETA_HE_random_Normal
####BETA_TIME_TRAIN_random_Norm
####BETA_TIME_SM_random_Norm





#Utility functions
TRAIN_SP = ASC_TRAIN                                      + \
     BETA_COST_random             * TRAIN_COST_SCALED + \
     BETA_TIME_TRAIN_random_Norm       * TRAIN_TT_SCALED   + \
     BETA_HE_random_Norm               * TRAIN_HE_SCALED           + \
     B_AGE_ADULTS_TRAIN * AGE_2                     + \
     B_AGE_3_TRAIN      * AGE_3                     + \
     B_AGE_ADULTS_TRAIN * AGE_4                     + \
     B_AGE_5_TRAIN      * AGE_5                     + \
     B_LUGGAGE_TRAIN    * LUGGAGE                   + \
     B_MALE_TRAIN       * MALE                      + \
     B_GA_TRAIN         * GA                        + \
     B_BUSINESS_TRAIN   * BUSINESS                  + \
     B_PRIVATE          * PRIVATE                   + \
     B_COMMUTER         * COMMUTER

SM_SP = ASC_SM                                   + \
     BETA_COST_random * SM_COST_SCALED + \
     BETA_TIME_SM_random_Norm       * SM_TT_SCALED   + \
     BETA_HE_random_Norm            * SM_HE_SCALED           + \
     B_AGE_ADULTS_SM * AGE_2                  + \
     B_AGE_ADULTS_SM * AGE_3                  + \
     B_AGE_4_SM      * AGE_4                  + \
     B_AGE_5_SM      * AGE_5                  + \
     B_FIRST_SM      * FIRST                  + \
     B_BUSINESS_SM   * BUSINESS               + \
     B_PRIVATE       * PRIVATE                + \
     B_COMMUTER      * COMMUTER

Car_SP = ASC_CAR                              + \
     BETA_COST_random     * CAR_COST_SCALED + \
     B_TIME_CAR * CAR_TT_SCALED



V = {3: Car_SP,1: TRAIN_SP,2: SM_SP}
av = {3: CAR_AV_SP,1: TRAIN_AV_SP,2: SM_AV}

vSeq.append(V)
ModNameSeqence.append("ML_"+str(len(vSeq)))
display(Markdown(f"# ^^^^^^^^^^^"))
display(Markdown(f"# {ModNameSeqence[-1]}"))

# ^^^^^^^^^^^

# ML_8

In [23]:
bestRes, results = estimate_N_compare(ModNameSeqence[-1]
                               , bestRes
                               , vSeq
                               , av
                               , CHOICE, Car_SP, TRAIN_SP, SM_SP, CAR_AV_SP, TRAIN_AV_SP, SM_AV
                               , database
                               , pandas
                               , myRandomNumberGenerators
                               , results
                               , ModNameSeqence=ModNameSeqence
                               , Draws=Draws)
best.append(bestRes)

## Estimate  ML_8 and compare with ML_7

#### Unrestricted model ML_8 and Restricted model ML_7

# Results model ML_8

                              Value        Std err         t-test  p-value
ASC_SM                     0.711020   0.000000e+00  1.797693e+308      0.0
ASC_TRAIN                  1.094947   0.000000e+00  1.797693e+308      0.0
BETA_COST_mean             1.008916  -0.000000e+00  1.797693e+308      0.0
BETA_COST_std              1.628410   0.000000e+00  1.797693e+308      0.0
BETA_HE_mean_Norm         -1.650951   0.000000e+00  1.797693e+308      0.0
BETA_HE_std_Norm           1.748922   0.000000e+00  1.797693e+308      0.0
BETA_TIME_SM_mean_Norm    -4.250008   0.000000e+00  1.797693e+308      0.0
BETA_TIME_SM_std_Norm      3.039631   0.000000e+00  1.797693e+308      0.0
BETA_TIME_TRAIN_mean_Norm -5.463609   0.000000e+00  1.797693e+308      0.0
BETA_TIME_TRAIN_std_Norm   2.102472   0.000000e+00  1.797693e+308      0.0
B_AGE_3_TRAIN             -1.570144  5.273843e-155 -2.977229e+154      0.0
B_AGE_4_SM                -2.658594  5.273843e-155 -5.041094e+154      0.0
B_AGE_5_SM               

# Results model ML_7

...

#### LogLikelihood TEST None disagrees with BenAkivaSwaitHorowitzTest ML_7

### TRY  CROSS VALIDATION

# Perform  Cross Validation Between ['ML_8', 'ML_7']

Model 0
Fold 0
Model 0
Fold 1
Model 0
Fold 2
Model 0
Fold 3
Model 0
Fold 4
Model 1
Fold 0
Model 1
Fold 1
Model 1
Fold 2
Model 1
Fold 3
Model 1
Fold 4
loglikelihood model ML_8: -1551.4422763239866 +/- 26.32265691974744
loglikelihood model ML_7: -1554.779627778157 +/- 54.8086239034275


## Accept ML_8 performs better off the sample

In [24]:
####BETA_COST_random
####BETA_HE_random_Normal
####BETA_TIME_TRAIN_random_Norm
####BETA_TIME_SM_random
####BETA_TIME_CAR_random





#Utility functions
TRAIN_SP = ASC_TRAIN                                      + \
     BETA_COST_random             * TRAIN_COST_SCALED + \
     BETA_TIME_TRAIN_random_Norm       * TRAIN_TT_SCALED   + \
     BETA_HE_random_Norm               * TRAIN_HE_SCALED           + \
     B_AGE_ADULTS_TRAIN * AGE_2                     + \
     B_AGE_3_TRAIN      * AGE_3                     + \
     B_AGE_ADULTS_TRAIN * AGE_4                     + \
     B_AGE_5_TRAIN      * AGE_5                     + \
     B_LUGGAGE_TRAIN    * LUGGAGE                   + \
     B_MALE_TRAIN       * MALE                      + \
     B_GA_TRAIN         * GA                        + \
     B_BUSINESS_TRAIN   * BUSINESS                  + \
     B_PRIVATE          * PRIVATE                   + \
     B_COMMUTER         * COMMUTER

SM_SP = ASC_SM                                   + \
     BETA_COST_random * SM_COST_SCALED + \
     BETA_TIME_SM_random       * SM_TT_SCALED   + \
     BETA_HE_random_Norm            * SM_HE_SCALED           + \
     B_AGE_ADULTS_SM * AGE_2                  + \
     B_AGE_ADULTS_SM * AGE_3                  + \
     B_AGE_4_SM      * AGE_4                  + \
     B_AGE_5_SM      * AGE_5                  + \
     B_FIRST_SM      * FIRST                  + \
     B_BUSINESS_SM   * BUSINESS               + \
     B_PRIVATE       * PRIVATE                + \
     B_COMMUTER      * COMMUTER

Car_SP = ASC_CAR                              + \
     BETA_COST_random     * CAR_COST_SCALED + \
     BETA_TIME_CAR_random * CAR_TT_SCALED



V = {3: Car_SP,1: TRAIN_SP,2: SM_SP}
av = {3: CAR_AV_SP,1: TRAIN_AV_SP,2: SM_AV}

vSeq.append(V)
ModNameSeqence.append("ML_"+str(len(vSeq)))
display(Markdown(f"# ^^^^^^^^^^^"))
display(Markdown(f"# {ModNameSeqence[-1]}"))

# ^^^^^^^^^^^

# ML_9

In [25]:
bestRes, results = estimate_N_compare(ModNameSeqence[-1]
                               , bestRes
                               , vSeq
                               , av
                               , CHOICE, Car_SP, TRAIN_SP, SM_SP, CAR_AV_SP, TRAIN_AV_SP, SM_AV
                               , database
                               , pandas
                               , myRandomNumberGenerators
                               , results
                               , ModNameSeqence=ModNameSeqence
                               , Draws=Draws)
best.append(bestRes)

## Estimate  ML_9 and compare with ML_8

#### Unrestricted model ML_9 and Restricted model ML_8

# Results model ML_9

                              Value        Std err         t-test  p-value
ASC_SM                    -0.971429  1.797693e+308 -5.403753e-309      1.0
ASC_TRAIN                 -0.989133  1.797693e+308 -5.502234e-309      1.0
BETA_COST_mean             1.275194  1.797693e+308  7.093503e-309      1.0
BETA_COST_std              1.412363  1.797693e+308  7.856531e-309      1.0
BETA_HE_mean_Norm         -1.345862  1.797693e+308 -7.486607e-309      1.0
BETA_HE_std_Norm           1.175220  1.797693e+308  6.537380e-309      1.0
BETA_TIME_CAR_mean         1.673371  1.797693e+308  9.308435e-309      1.0
BETA_TIME_CAR_std          0.556150  1.797693e+308  3.093688e-309      1.0
BETA_TIME_SM_mean          1.446401  1.797693e+308  8.045870e-309      1.0
BETA_TIME_SM_std           0.186514  1.797693e+308  1.037521e-309      1.0
BETA_TIME_TRAIN_mean_Norm -4.770184  1.797693e+308 -2.653503e-308      1.0
BETA_TIME_TRAIN_std_Norm   1.755904  1.797693e+308  9.767540e-309      1.0
B_AGE_3_TRAIN            

# Results model ML_8

...

### Diff. rho bar square ML_8 > ML_9 <- False

***************
RatioTest 264.0403939161606
Critical 3.841458820694124
Chose Unrestricted (reject H0 the parameters of the unrestricted model are 0)


### LL Ratio Test Result (abs(STAT)-CHI2)260.1989350954665

Probability that the unrestricted model is false having greater rho bar squere, is below threshold:
Threshold:0.07
Probability:1.9804506898925944e-172
Difference:-0.07


#### LogLikelihood ML_9 - BenAkivaSwaitHorowitzTest ML_9

In [26]:
####BETA_COST_random
####BETA_HE_random_Normal
####BETA_TIME_TRAIN_random_Norm
####BETA_TIME_SM_random
####BETA_TIME_CAR_random_Norm





#Utility functions
TRAIN_SP = ASC_TRAIN                                      + \
     BETA_COST_random             * TRAIN_COST_SCALED + \
     BETA_TIME_TRAIN_random_Norm       * TRAIN_TT_SCALED   + \
     BETA_HE_random_Norm               * TRAIN_HE_SCALED           + \
     B_AGE_ADULTS_TRAIN * AGE_2                     + \
     B_AGE_3_TRAIN      * AGE_3                     + \
     B_AGE_ADULTS_TRAIN * AGE_4                     + \
     B_AGE_5_TRAIN      * AGE_5                     + \
     B_LUGGAGE_TRAIN    * LUGGAGE                   + \
     B_MALE_TRAIN       * MALE                      + \
     B_GA_TRAIN         * GA                        + \
     B_BUSINESS_TRAIN   * BUSINESS                  + \
     B_PRIVATE          * PRIVATE                   + \
     B_COMMUTER         * COMMUTER

SM_SP = ASC_SM                                   + \
     BETA_COST_random * SM_COST_SCALED + \
     BETA_TIME_SM_random       * SM_TT_SCALED   + \
     BETA_HE_random_Norm            * SM_HE_SCALED           + \
     B_AGE_ADULTS_SM * AGE_2                  + \
     B_AGE_ADULTS_SM * AGE_3                  + \
     B_AGE_4_SM      * AGE_4                  + \
     B_AGE_5_SM      * AGE_5                  + \
     B_FIRST_SM      * FIRST                  + \
     B_BUSINESS_SM   * BUSINESS               + \
     B_PRIVATE       * PRIVATE                + \
     B_COMMUTER      * COMMUTER

Car_SP = ASC_CAR                              + \
     BETA_COST_random     * CAR_COST_SCALED + \
     BETA_TIME_CAR_random_Norm * CAR_TT_SCALED



V = {3: Car_SP,1: TRAIN_SP,2: SM_SP}
av = {3: CAR_AV_SP,1: TRAIN_AV_SP,2: SM_AV}

vSeq.append(V)
ModNameSeqence.append("ML_"+str(len(vSeq)))
display(Markdown(f"# ^^^^^^^^^^^"))
display(Markdown(f"# {ModNameSeqence[-1]}"))

# ^^^^^^^^^^^

# ML_10

In [27]:
bestRes, results = estimate_N_compare(ModNameSeqence[-1]
                               , bestRes
                               , vSeq
                               , av
                               , CHOICE, Car_SP, TRAIN_SP, SM_SP, CAR_AV_SP, TRAIN_AV_SP, SM_AV
                               , database
                               , pandas
                               , myRandomNumberGenerators
                               , results
                               , ModNameSeqence=ModNameSeqence
                               , Draws=Draws)
best.append(bestRes)

## Estimate  ML_10 and compare with ML_9

#### Unrestricted model ML_10 and Restricted model ML_9

# Results model ML_10

                              Value        Std err         t-test  p-value
ASC_SM                     0.005850  1.797693e+308  3.254237e-311      1.0
ASC_TRAIN                  0.416358  1.797693e+308  2.316069e-309      1.0
BETA_COST_mean             1.303010  1.797693e+308  7.248236e-309      1.0
BETA_COST_std              1.115152  1.797693e+308  6.203239e-309      1.0
BETA_HE_mean_Norm         -1.431921  1.797693e+308 -7.965324e-309      1.0
BETA_HE_std_Norm           1.230198  1.797693e+308  6.843205e-309      1.0
BETA_TIME_CAR_mean_Norm   -5.161364  1.797693e+308 -2.871104e-308      1.0
BETA_TIME_CAR_std_Norm     2.342177  1.797693e+308  1.302879e-308      1.0
BETA_TIME_SM_mean          1.535906  1.797693e+308  8.543760e-309      1.0
BETA_TIME_SM_std           0.412692  1.797693e+308  2.295675e-309      1.0
BETA_TIME_TRAIN_mean_Norm -5.081586  1.797693e+308 -2.826726e-308      1.0
BETA_TIME_TRAIN_std_Norm   1.904084  1.797693e+308  1.059182e-308      1.0
B_AGE_3_TRAIN            

# Results model ML_9

...

#### LogLikelihood TEST None disagrees with BenAkivaSwaitHorowitzTest ML_10

### TRY  CROSS VALIDATION

# Perform  Cross Validation Between ['ML_10', 'ML_9']

Model 0
Fold 0
Model 0
Fold 1
Model 0
Fold 2
Model 0
Fold 3
Model 0
Fold 4
Model 1
Fold 0
Model 1
Fold 1
Model 1
Fold 2
Model 1
Fold 3
Model 1
Fold 4
loglikelihood model ML_10: -1560.766998215737 +/- 46.57429200753703
loglikelihood model ML_9: -1561.7921517591872 +/- 45.78976748469738


## Accept ML_10 performs better off the sample

In [29]:
####q_COST
####BETA_COST_random
####BETA_HE_random_Norm
####BETA_TIME_TRAIN_random_Norm
####BETA_TIME_SM_random
####BETA_TIME_CAR_random_Norm





#Utility functions
TRAIN_SP = ASC_TRAIN                                      + \
     BETA_COST_random             * TRAIN_COST_SCALED**q_COST + \
     BETA_TIME_TRAIN_random_Norm       * TRAIN_TT_SCALED   + \
     BETA_HE_random_Norm               * TRAIN_HE_SCALED           + \
     B_AGE_ADULTS_TRAIN * AGE_2                     + \
     B_AGE_3_TRAIN      * AGE_3                     + \
     B_AGE_ADULTS_TRAIN * AGE_4                     + \
     B_AGE_5_TRAIN      * AGE_5                     + \
     B_LUGGAGE_TRAIN    * LUGGAGE                   + \
     B_MALE_TRAIN       * MALE                      + \
     B_GA_TRAIN         * GA                        + \
     B_BUSINESS_TRAIN   * BUSINESS                  + \
     B_PRIVATE          * PRIVATE                   + \
     B_COMMUTER         * COMMUTER

SM_SP = ASC_SM                                   + \
     BETA_COST_random * SM_COST_SCALED**q_COST + \
     BETA_TIME_SM_random       * SM_TT_SCALED   + \
     BETA_HE_random_Norm            * SM_HE_SCALED           + \
     B_AGE_ADULTS_SM * AGE_2                  + \
     B_AGE_ADULTS_SM * AGE_3                  + \
     B_AGE_4_SM      * AGE_4                  + \
     B_AGE_5_SM      * AGE_5                  + \
     B_FIRST_SM      * FIRST                  + \
     B_BUSINESS_SM   * BUSINESS               + \
     B_PRIVATE       * PRIVATE                + \
     B_COMMUTER      * COMMUTER

Car_SP = ASC_CAR                              + \
     BETA_COST_random     * CAR_COST_SCALED**q_COST + \
     BETA_TIME_CAR_random_Norm * CAR_TT_SCALED



V = {3: Car_SP,1: TRAIN_SP,2: SM_SP}
av = {3: CAR_AV_SP,1: TRAIN_AV_SP,2: SM_AV}

vSeq.append(V)
ModNameSeqence.append("ML_"+str(len(vSeq)))
display(Markdown(f"# ^^^^^^^^^^^"))
display(Markdown(f"# {ModNameSeqence[-1]}"))

# ^^^^^^^^^^^

# ML_11

In [30]:
bestRes, results = estimate_N_compare(ModNameSeqence[-1]
                               , bestRes
                               , vSeq
                               , av
                               , CHOICE, Car_SP, TRAIN_SP, SM_SP, CAR_AV_SP, TRAIN_AV_SP, SM_AV
                               , database
                               , pandas
                               , myRandomNumberGenerators
                               , results
                               , ModNameSeqence=ModNameSeqence
                               , Draws=Draws)
best.append(bestRes)

## Estimate  ML_11 and compare with ML_10

#### Unrestricted model ML_11 and Restricted model ML_10

# Results model ML_11

                              Value        Std err         t-test  p-value
ASC_SM                    -1.087554  3.925442e-156 -2.770526e+155      0.0
ASC_TRAIN                 -1.322085  3.925442e-156 -3.367989e+155      0.0
BETA_COST_mean             1.712237  3.925442e-156  4.361894e+155      0.0
BETA_COST_std              1.037126  3.925442e-156  2.642062e+155      0.0
BETA_HE_mean_Norm         -1.321791  3.925442e-156 -3.367240e+155      0.0
BETA_HE_std_Norm           1.076659  3.925442e-156  2.742772e+155      0.0
BETA_TIME_CAR_mean_Norm   -5.122447  3.925442e-156 -1.304935e+156      0.0
BETA_TIME_CAR_std_Norm     2.186445  3.925442e-156  5.569932e+155      0.0
BETA_TIME_SM_mean          1.573774  3.925442e-156  4.009164e+155      0.0
BETA_TIME_SM_std           0.401510  3.925442e-156  1.022841e+155      0.0
BETA_TIME_TRAIN_mean_Norm -4.987347  3.925442e-156 -1.270518e+156      0.0
BETA_TIME_TRAIN_std_Norm   1.857135  3.925442e-156  4.731021e+155      0.0
B_AGE_3_TRAIN            

# Results model ML_10

...

### Diff. rho bar square ML_10 > ML_11 <- True

***************
RatioTest 0.34787728761148173
Critical 3.841458820694124
Chose Restricted (fail to reject H0 the parameters of the unrestricted model are 0)


### LL ratio test result -3.4935815330826423

Probability that the restricted model is false having greater rho bar squere, is below threshold:
Threshold:0.07
Probability:0.011176373878000075
Difference:-0.05882362612199993


#### LogLikelihood ML_10 - BenAkivaSwaitHorowitzTest ML_10

In [31]:
####q_COST
####q_TIME
####BETA_COST_random
####BETA_HE_random_Norm
####BETA_TIME_TRAIN_random_Norm
####BETA_TIME_SM_random
####BETA_TIME_CAR





#Utility functions
TRAIN_SP = ASC_TRAIN                                      + \
     BETA_COST_random             * TRAIN_COST_SCALED**q_COST + \
     BETA_TIME_TRAIN_random_Norm       * TRAIN_TT_SCALED**q_TIME   + \
     BETA_HE_random_Norm               * TRAIN_HE_SCALED           + \
     B_AGE_ADULTS_TRAIN * AGE_2                     + \
     B_AGE_3_TRAIN      * AGE_3                     + \
     B_AGE_ADULTS_TRAIN * AGE_4                     + \
     B_AGE_5_TRAIN      * AGE_5                     + \
     B_LUGGAGE_TRAIN    * LUGGAGE                   + \
     B_MALE_TRAIN       * MALE                      + \
     B_GA_TRAIN         * GA                        + \
     B_BUSINESS_TRAIN   * BUSINESS                  + \
     B_PRIVATE          * PRIVATE                   + \
     B_COMMUTER         * COMMUTER

SM_SP = ASC_SM                                   + \
     BETA_COST_random * SM_COST_SCALED**q_COST + \
     BETA_TIME_SM_random       * SM_TT_SCALED**q_TIME   + \
     BETA_HE_random_Norm            * SM_HE_SCALED           + \
     B_AGE_ADULTS_SM * AGE_2                  + \
     B_AGE_ADULTS_SM * AGE_3                  + \
     B_AGE_4_SM      * AGE_4                  + \
     B_AGE_5_SM      * AGE_5                  + \
     B_FIRST_SM      * FIRST                  + \
     B_BUSINESS_SM   * BUSINESS               + \
     B_PRIVATE       * PRIVATE                + \
     B_COMMUTER      * COMMUTER

Car_SP = ASC_CAR                              + \
     BETA_COST_random     * CAR_COST_SCALED**q_COST + \
     B_TIME_CAR * CAR_TT_SCALED**q_TIME



V = {3: Car_SP,1: TRAIN_SP,2: SM_SP}
av = {3: CAR_AV_SP,1: TRAIN_AV_SP,2: SM_AV}

vSeq.append(V)
ModNameSeqence.append("ML_"+str(len(vSeq)))

display(Markdown(f"# ^^^^^^^^^^^"))
display(Markdown(f"# {ModNameSeqence[-1]}"))

# ^^^^^^^^^^^

# ML_12

In [32]:
bestRes, results = estimate_N_compare(ModNameSeqence[-1]
                               , bestRes
                               , vSeq
                               , av
                               , CHOICE, Car_SP, TRAIN_SP, SM_SP, CAR_AV_SP, TRAIN_AV_SP, SM_AV
                               , database
                               , pandas
                               , myRandomNumberGenerators
                               , results
                               , ModNameSeqence=ModNameSeqence
                               , Draws=Draws)
best.append(bestRes)

## Estimate  ML_12 and compare with ML_10

#### Unrestricted model ML_12 and Restricted model ML_10

# Results model ML_12

                               Value        Std err         t-test  p-value
ASC_SM                     -4.436838  1.797693e+308 -2.468073e-308      1.0
ASC_TRAIN                   4.854622  1.797693e+308  2.700473e-308      1.0
BETA_COST_mean              1.888467  1.797693e+308  1.050495e-308      1.0
BETA_COST_std               1.238315  1.797693e+308  6.888357e-309      1.0
BETA_HE_mean_Norm          -1.452465  1.797693e+308 -8.079602e-309      1.0
BETA_HE_std_Norm            1.300490  1.797693e+308  7.234215e-309      1.0
BETA_TIME_SM_mean           3.044952  1.797693e+308  1.693811e-308      1.0
BETA_TIME_SM_std            0.143491  1.797693e+308  7.981974e-310      1.0
BETA_TIME_TRAIN_mean_Norm -28.948553  1.797693e+308 -1.610317e-307      1.0
BETA_TIME_TRAIN_std_Norm    2.841075  1.797693e+308  1.580401e-308      1.0
B_AGE_3_TRAIN              -3.229876  -0.000000e+00  1.797693e+308      0.0
B_AGE_4_SM                 -0.976249  -0.000000e+00  1.797693e+308      0.0
B_AGE_5_SM  

# Results model ML_10

...

### Diff. rho bar square ML_10 > ML_12 <- False

***************
RatioTest 55.05891765794877
Critical 3.841458820694124
Chose Unrestricted (reject H0 the parameters of the unrestricted model are 0)


### LL Ratio Test Result (abs(STAT)-CHI2)51.21745883725465

Probability that the unrestricted model is false having greater rho bar squere, is below threshold:
Threshold:0.07
Probability:0.0
Difference:-0.07


#### LogLikelihood ML_12 - BenAkivaSwaitHorowitzTest ML_12

In [33]:
####q_COST
####q_TIME
####q_HE
####BETA_COST_random
####BETA_HE_random_Norm
####BETA_TIME_TRAIN_random_Norm
####BETA_TIME_SM_random
####BETA_TIME_CAR


#Utility functions
TRAIN_SP = ASC_TRAIN                                      + \
     BETA_COST_random             * TRAIN_COST_SCALED**q_COST + \
     BETA_TIME_TRAIN_random_Norm       * TRAIN_TT_SCALED**q_TIME   + \
     BETA_HE_random_Norm               * TRAIN_HE_SCALED**q_HE           + \
     B_AGE_ADULTS_TRAIN * AGE_2                     + \
     B_AGE_3_TRAIN      * AGE_3                     + \
     B_AGE_ADULTS_TRAIN * AGE_4                     + \
     B_AGE_5_TRAIN      * AGE_5                     + \
     B_LUGGAGE_TRAIN    * LUGGAGE                   + \
     B_MALE_TRAIN       * MALE                      + \
     B_GA_TRAIN         * GA                        + \
     B_BUSINESS_TRAIN   * BUSINESS                  + \
     B_PRIVATE          * PRIVATE                   + \
     B_COMMUTER         * COMMUTER

SM_SP = ASC_SM                                   + \
     BETA_COST_random * SM_COST_SCALED**q_COST + \
     BETA_TIME_SM_random       * SM_TT_SCALED**q_TIME   + \
     BETA_HE_random_Norm            * SM_HE_SCALED**q_HE           + \
     B_AGE_ADULTS_SM * AGE_2                  + \
     B_AGE_ADULTS_SM * AGE_3                  + \
     B_AGE_4_SM      * AGE_4                  + \
     B_AGE_5_SM      * AGE_5                  + \
     B_FIRST_SM      * FIRST                  + \
     B_BUSINESS_SM   * BUSINESS               + \
     B_PRIVATE       * PRIVATE                + \
     B_COMMUTER      * COMMUTER

Car_SP = ASC_CAR                              + \
     BETA_COST_random     * CAR_COST_SCALED**q_COST + \
     B_TIME_CAR * CAR_TT_SCALED**q_TIME



V = {3: Car_SP,1: TRAIN_SP,2: SM_SP}
av = {3: CAR_AV_SP,1: TRAIN_AV_SP,2: SM_AV}

vSeq.append(V)
ModNameSeqence.append("ML_"+str(len(vSeq)))
print(ModNameSeqence[-1])

ML_13


In [34]:
bestRes, results = estimate_N_compare(ModNameSeqence[-1]
                               , bestRes
                               , vSeq
                               , av
                               , CHOICE, Car_SP, TRAIN_SP, SM_SP, CAR_AV_SP, TRAIN_AV_SP, SM_AV
                               , database
                               , pandas
                               , myRandomNumberGenerators
                               , results
                               , ModNameSeqence=ModNameSeqence
                               , Draws=Draws)
best.append(bestRes)

## Estimate  ML_13 and compare with ML_12

#### Unrestricted model ML_13 and Restricted model ML_12

# Results model ML_13

                               Value        Std err         t-test  p-value
ASC_SM                     65.478482  1.797693e+308  3.642361e-307      1.0
ASC_TRAIN                  69.857465  1.797693e+308  3.885951e-307      1.0
BETA_COST_mean              2.057064  1.797693e+308  1.144280e-308      1.0
BETA_COST_std               1.257500  1.797693e+308  6.995075e-309      1.0
BETA_HE_mean_Norm         -73.310530  1.797693e+308 -4.078034e-307      1.0
BETA_HE_std_Norm            3.340594  1.797693e+308  1.858267e-308      1.0
BETA_TIME_SM_mean           2.721761  1.797693e+308  1.514030e-308      1.0
BETA_TIME_SM_std            0.100922  1.797693e+308  5.613960e-310      1.0
BETA_TIME_TRAIN_mean_Norm -18.714603  1.797693e+308 -1.041034e-307      1.0
BETA_TIME_TRAIN_std_Norm    2.140489  1.797693e+308  1.190686e-308      1.0
B_AGE_3_TRAIN              -3.239318  -0.000000e+00  1.797693e+308      0.0
B_AGE_4_SM                 -2.256623  -0.000000e+00  1.797693e+308      0.0
B_AGE_5_SM  

# Results model ML_12

...

### Diff. rho bar square ML_12 > ML_13 <- False

***************
RatioTest 194.1407348983248
Critical 3.841458820694124
Chose Unrestricted (reject H0 the parameters of the unrestricted model are 0)


### LL Ratio Test Result (abs(STAT)-CHI2)190.29927607763068

Probability that the unrestricted model is false having greater rho bar squere, is below threshold:
Threshold:0.07
Probability:4.027962062765011e-49
Difference:-0.07


#### LogLikelihood ML_13 - BenAkivaSwaitHorowitzTest ML_13

In [35]:
####q_COST
####q_TIME
####q_HE
####BETA_COST_random
####BETA_HE_random_Norm
####BETA_TIME_TRAIN_random_Norm
####BETA_TIME_SM_random
####BETA_TIME_CAR_random_Norm


#Utility functions
TRAIN_SP = ASC_TRAIN                                      + \
     BETA_COST_random             * TRAIN_COST_SCALED**q_COST + \
     BETA_TIME_TRAIN_random_Norm       * TRAIN_TT_SCALED**q_TIME   + \
     BETA_HE_random_Norm               * TRAIN_HE_SCALED**q_HE           + \
     B_AGE_ADULTS_TRAIN * AGE_2                     + \
     B_AGE_3_TRAIN      * AGE_3                     + \
     B_AGE_ADULTS_TRAIN * AGE_4                     + \
     B_AGE_5_TRAIN      * AGE_5                     + \
     B_LUGGAGE_TRAIN    * LUGGAGE                   + \
     B_MALE_TRAIN       * MALE                      + \
     B_GA_TRAIN         * GA                        + \
     B_BUSINESS_TRAIN   * BUSINESS                  + \
     B_PRIVATE          * PRIVATE                   + \
     B_COMMUTER         * COMMUTER

SM_SP = ASC_SM                                   + \
     BETA_COST_random * SM_COST_SCALED**q_COST + \
     BETA_TIME_SM_random       * SM_TT_SCALED**q_TIME   + \
     BETA_HE_random_Norm            * SM_HE_SCALED**q_HE           + \
     B_AGE_ADULTS_SM * AGE_2                  + \
     B_AGE_ADULTS_SM * AGE_3                  + \
     B_AGE_4_SM      * AGE_4                  + \
     B_AGE_5_SM      * AGE_5                  + \
     B_FIRST_SM      * FIRST                  + \
     B_BUSINESS_SM   * BUSINESS               + \
     B_PRIVATE       * PRIVATE                + \
     B_COMMUTER      * COMMUTER

Car_SP = ASC_CAR                              + \
     BETA_COST_random     * CAR_COST_SCALED**q_COST + \
     BETA_TIME_CAR_random_Norm * CAR_TT_SCALED**q_TIME



V = {3: Car_SP,1: TRAIN_SP,2: SM_SP}
av = {3: CAR_AV_SP,1: TRAIN_AV_SP,2: SM_AV}

vSeq.append(V)
ModNameSeqence.append("ML_"+str(len(vSeq)))
print(ModNameSeqence[-1])

ML_14


In [ ]:
bestRes, results = estimate_N_compare(ModNameSeqence[-1]
                               , bestRes
                               , vSeq
                               , av
                               , CHOICE, Car_SP, TRAIN_SP, SM_SP, CAR_AV_SP, TRAIN_AV_SP, SM_AV
                               , database
                               , pandas
                               , myRandomNumberGenerators
                               , results
                               , ModNameSeqence=ModNameSeqence
                               , Draws=Draws)
best.append(bestRes)

## Estimate  ML_14 and compare with ML_13

#### Unrestricted model ML_14 and Restricted model ML_13

# Results model ML_14

                              Value        Std err         t-test  p-value
ASC_SM                     0.321295  1.797693e+308  1.787262e-309      1.0
ASC_TRAIN                 -0.352301  1.797693e+308 -1.959739e-309      1.0
BETA_COST_mean             0.862195  1.797693e+308  4.796118e-309      1.0
BETA_COST_std              1.715287  1.797693e+308  9.541603e-309      1.0
BETA_HE_mean_Norm         -0.779271  1.797693e+308 -4.334839e-309      1.0
BETA_HE_std_Norm           1.091373  1.797693e+308  6.070962e-309      1.0
BETA_TIME_CAR_mean_Norm   -1.691893  1.797693e+308 -9.411465e-309      1.0
BETA_TIME_CAR_std_Norm     1.048030  1.797693e+308  5.829863e-309      1.0
BETA_TIME_SM_mean          0.318845  1.797693e+308  1.773636e-309      1.0
BETA_TIME_SM_std           1.347796  1.797693e+308  7.497362e-309      1.0
BETA_TIME_TRAIN_mean_Norm -2.121797  1.797693e+308 -1.180289e-308      1.0
BETA_TIME_TRAIN_std_Norm   1.759655  1.797693e+308  9.788408e-309      1.0
B_AGE_3_TRAIN            

# Results model ML_13

...

#### LogLikelihood ML_14 - BenAkivaSwaitHorowitzTest ML_13

#### LogLikelihood TEST ML_14 disagrees with BenAkivaSwaitHorowitzTest ML_13

### TRY  CROSS VALIDATION

# Perform  Cross Validation Between ['ML_14', 'ML_13']

Model 0
Fold 0
Model 0
Fold 1
Model 0
Fold 2
Model 0
Fold 3
Model 0
Fold 4
Model 1
Fold 0
Model 1
Fold 1
Model 1
Fold 2


In [ ]:
####BETA_COST_random
####BETA_HE_random_Normal
####BETA_TIME_TRAIN_random_Norm
####BETA_TIME_SM_random
####BETA_TIME_CAR_random_Norm

####q_TIME




#Utility functions
TRAIN_SP = ASC_TRAIN                                      + \
     BETA_COST_random             * TRAIN_COST_SCALED + \
     BETA_TIME_TRAIN_random_Norm       * TRAIN_TT_SCALED**q_TIME   + \
     BETA_HE_random_Norm               * TRAIN_HE_SCALED           + \
     B_AGE_ADULTS_TRAIN * AGE_2                     + \
     B_AGE_3_TRAIN      * AGE_3                     + \
     B_AGE_ADULTS_TRAIN * AGE_4                     + \
     B_AGE_5_TRAIN      * AGE_5                     + \
     B_LUGGAGE_TRAIN    * LUGGAGE                   + \
     B_MALE_TRAIN       * MALE                      + \
     B_GA_TRAIN         * GA                        + \
     B_BUSINESS_TRAIN   * BUSINESS                  + \
     B_PRIVATE          * PRIVATE                   + \
     B_COMMUTER         * COMMUTER

SM_SP = ASC_SM                                   + \
     BETA_COST_random * SM_COST_SCALED + \
     BETA_TIME_SM_random       * SM_TT_SCALED**q_TIME   + \
     BETA_HE_random_Norm            * SM_HE_SCALED           + \
     B_AGE_ADULTS_SM * AGE_2                  + \
     B_AGE_ADULTS_SM * AGE_3                  + \
     B_AGE_4_SM      * AGE_4                  + \
     B_AGE_5_SM      * AGE_5                  + \
     B_FIRST_SM      * FIRST                  + \
     B_BUSINESS_SM   * BUSINESS               + \
     B_PRIVATE       * PRIVATE                + \
     B_COMMUTER      * COMMUTER

Car_SP = ASC_CAR                              + \
     BETA_COST_random     * CAR_COST_SCALED + \
     BETA_TIME_CAR_random_Norm * CAR_TT_SCALED**q_TIME



V = {3: Car_SP,1: TRAIN_SP,2: SM_SP}
av = {3: CAR_AV_SP,1: TRAIN_AV_SP,2: SM_AV}

vSeq.append(V)
ModNameSeqence.append("ML_"+str(len(vSeq)))
display(Markdown(f"# ^^^^^^^^^^^"))
display(Markdown(f"# {ModNameSeqence[-1]}"))

In [ ]:
bestRes, results = estimate_N_compare(ModNameSeqence[-1]
                               , bestRes
                               , vSeq
                               , av
                               , CHOICE, Car_SP, TRAIN_SP, SM_SP, CAR_AV_SP, TRAIN_AV_SP, SM_AV
                               , database
                               , pandas
                               , myRandomNumberGenerators
                               , results
                               , ModNameSeqence=ModNameSeqence
                               , Draws=Draws)
best.append(bestRes)

In [ ]:
####BETA_COST_random
####BETA_HE_random_Normal
####BETA_TIME_TRAIN_random_Norm
####BETA_TIME_SM_random
####BETA_TIME_CAR_random_Norm

####q_TIME




#Utility functions
TRAIN_SP = ASC_TRAIN                                      + \
     BETA_COST_random             * TRAIN_COST_SCALED + \
     BETA_TIME_TRAIN_random_Norm       * TRAIN_TT_SCALED**q_TIME   + \
     BETA_HE_random_Norm               * TRAIN_HE_SCALED**q_HE           + \
     B_AGE_ADULTS_TRAIN * AGE_2                     + \
     B_AGE_3_TRAIN      * AGE_3                     + \
     B_AGE_ADULTS_TRAIN * AGE_4                     + \
     B_AGE_5_TRAIN      * AGE_5                     + \
     B_LUGGAGE_TRAIN    * LUGGAGE                   + \
     B_MALE_TRAIN       * MALE                      + \
     B_GA_TRAIN         * GA                        + \
     B_BUSINESS_TRAIN   * BUSINESS                  + \
     B_PRIVATE          * PRIVATE                   + \
     B_COMMUTER         * COMMUTER

SM_SP = ASC_SM                                   + \
     BETA_COST_random * SM_COST_SCALED + \
     BETA_TIME_SM_random       * SM_TT_SCALED**q_TIME   + \
     BETA_HE_random_Norm            * SM_HE_SCALED**q_HE           + \
     B_AGE_ADULTS_SM * AGE_2                  + \
     B_AGE_ADULTS_SM * AGE_3                  + \
     B_AGE_4_SM      * AGE_4                  + \
     B_AGE_5_SM      * AGE_5                  + \
     B_FIRST_SM      * FIRST                  + \
     B_BUSINESS_SM   * BUSINESS               + \
     B_PRIVATE       * PRIVATE                + \
     B_COMMUTER      * COMMUTER

Car_SP = ASC_CAR                              + \
     BETA_COST_random     * CAR_COST_SCALED + \
     BETA_TIME_CAR_random_Norm * CAR_TT_SCALED**q_TIME



V = {3: Car_SP,1: TRAIN_SP,2: SM_SP}
av = {3: CAR_AV_SP,1: TRAIN_AV_SP,2: SM_AV}

vSeq.append(V)
ModNameSeqence.append("ML_"+str(len(vSeq)))
display(Markdown(f"# ^^^^^^^^^^^"))
display(Markdown(f"# {ModNameSeqence[-1]}"))

In [ ]:
bestRes, results = estimate_N_compare(ModNameSeqence[-1]
                               , bestRes
                               , vSeq
                               , av
                               , CHOICE, Car_SP, TRAIN_SP, SM_SP, CAR_AV_SP, TRAIN_AV_SP, SM_AV
                               , database
                               , pandas
                               , myRandomNumberGenerators
                               , results
                               , ModNameSeqence=ModNameSeqence
                               , Draws=Draws)
best.append(bestRes)

In [ ]:
####q_COST
####BETA_COST_random
####BETA_HE_random_Norm
####BETA_TIME_TRAIN_random_Norm
####BETA_TIME_SM_random
####BETA_TIME_CAR_random_Norm
####BETA_BUSINESS_TRAIN_random_Norm
####BETA_BUSINESS_SM_random_Norm





#Utility functions
TRAIN_SP = ASC_TRAIN                                      + \
     BETA_COST_random             * TRAIN_COST_SCALED + \
     BETA_TIME_TRAIN_random_Norm       * TRAIN_TT_SCALED   + \
     BETA_HE_random_Norm               * TRAIN_HE_SCALED           + \
     B_AGE_ADULTS_TRAIN * AGE_2                     + \
     B_AGE_3_TRAIN      * AGE_3                     + \
     B_AGE_ADULTS_TRAIN * AGE_4                     + \
     B_AGE_5_TRAIN      * AGE_5                     + \
     B_LUGGAGE_TRAIN    * LUGGAGE                   + \
     B_MALE_TRAIN       * MALE                      + \
     B_GA_TRAIN         * GA                        + \
     BETA_BUSINESS_TRAIN_random_Norm   * BUSINESS                  + \
     B_PRIVATE          * PRIVATE                   + \
     B_COMMUTER         * COMMUTER

SM_SP = ASC_SM                                   + \
     BETA_COST_random * SM_COST_SCALED + \
     BETA_TIME_SM_random       * SM_TT_SCALED   + \
     BETA_HE_random_Norm            * SM_HE_SCALED           + \
     B_AGE_ADULTS_SM * AGE_2                  + \
     B_AGE_ADULTS_SM * AGE_3                  + \
     B_AGE_4_SM      * AGE_4                  + \
     B_AGE_5_SM      * AGE_5                  + \
     B_FIRST_SM      * FIRST                  + \
     BETA_BUSINESS_SM_random_Norm   * BUSINESS               + \
     B_PRIVATE       * PRIVATE                + \
     B_COMMUTER      * COMMUTER

Car_SP = ASC_CAR                              + \
     BETA_COST_random     * CAR_COST_SCALED + \
     BETA_TIME_CAR_random_Norm * CAR_TT_SCALED



V = {3: Car_SP,1: TRAIN_SP,2: SM_SP}
av = {3: CAR_AV_SP,1: TRAIN_AV_SP,2: SM_AV}

vSeq.append(V)
ModNameSeqence.append("ML_"+str(len(vSeq)))
display(Markdown(f"# ^^^^^^^^^^^"))
display(Markdown(f"# {ModNameSeqence[-1]}"))

In [ ]:
bestRes, results = estimate_N_compare(ModNameSeqence[-1]
                               , bestRes
                               , vSeq
                               , av
                               , CHOICE, Car_SP, TRAIN_SP, SM_SP, CAR_AV_SP, TRAIN_AV_SP, SM_AV
                               , database
                               , pandas
                               , myRandomNumberGenerators
                               , results
                               , ModNameSeqence=ModNameSeqence
                               , Draws=Draws)
best.append(bestRes)

# Exponential distributions definied in negative domain
## final run with 1000 draws

In [ ]:
####q_COST
####q_TIME
####q_HE
####BETA_COST_random
####BETA_HE_random_Norm
####BETA_TIME_TRAIN_random_Norm
####BETA_TIME_SM_random
####BETA_TIME_CAR


#Utility functions
TRAIN_SP = ASC_TRAIN                                      + \
     BETA_COST_random             * TRAIN_COST_SCALED**q_COST + \
     BETA_TIME_TRAIN_random_Norm       * TRAIN_TT_SCALED**q_TIME   + \
     BETA_HE_random_Norm               * TRAIN_HE_SCALED**q_HE           + \
     B_AGE_ADULTS_TRAIN * AGE_2                     + \
     B_AGE_3_TRAIN      * AGE_3                     + \
     B_AGE_ADULTS_TRAIN * AGE_4                     + \
     B_AGE_5_TRAIN      * AGE_5                     + \
     B_LUGGAGE_TRAIN    * LUGGAGE                   + \
     B_MALE_TRAIN       * MALE                      + \
     B_GA_TRAIN         * GA                        + \
     B_BUSINESS_TRAIN   * BUSINESS                  + \
     B_PRIVATE          * PRIVATE                   + \
     B_COMMUTER         * COMMUTER

SM_SP = ASC_SM                                   + \
     BETA_COST_random * SM_COST_SCALED**q_COST + \
     BETA_TIME_SM_random       * SM_TT_SCALED**q_TIME   + \
     BETA_HE_random_Norm            * SM_HE_SCALED**q_HE           + \
     B_AGE_ADULTS_SM * AGE_2                  + \
     B_AGE_ADULTS_SM * AGE_3                  + \
     B_AGE_4_SM      * AGE_4                  + \
     B_AGE_5_SM      * AGE_5                  + \
     B_FIRST_SM      * FIRST                  + \
     B_BUSINESS_SM   * BUSINESS               + \
     B_PRIVATE       * PRIVATE                + \
     B_COMMUTER      * COMMUTER

Car_SP = ASC_CAR                              + \
     BETA_COST_random     * CAR_COST_SCALED**q_COST + \
     B_TIME_CAR * CAR_TT_SCALED**q_TIME



V = {3: Car_SP,1: TRAIN_SP,2: SM_SP}
av = {3: CAR_AV_SP,1: TRAIN_AV_SP,2: SM_AV}

vSeq.append(V)
ModNameSeqence.append("ML_"+str(len(vSeq)))
print(ModNameSeqence[-1])

In [ ]:
best.append(estimate_N_compare(ModNameSeqence[-1], best[-1], ModNameSeqence=ModNameSeqence, Draws=1000))

In [ ]:
bestRes, results = estimate_N_compare(ModNameSeqence[-1]
                               , bestRes
                               , vSeq
                               , av
                               , CHOICE, Car_SP, TRAIN_SP, SM_SP, CAR_AV_SP, TRAIN_AV_SP, SM_AV
                               , database
                               , pandas
                               , myRandomNumberGenerators
                               , results
                               , ModNameSeqence=ModNameSeqence
                               , Draws=1000)
best.append(bestRes)

In [ ]:
parN = []
LL = []
robar2 = []

for j,m in enumerate(ModNameSeqence):
    #best.append(estimate_N_compare(m, best[-1], ModNameSeqence=ModNameSeqence, threshold=0.07, testOnly=True))
    
    i = ModNameSeqence.index(best[j])
    
    LL.append(results[i].data.logLike)
    parN.append(len(results[i].data.betaNames))
    robar2.append(results[i].data.rhoBarSquare)

In [ ]:
"""
=======================
Plot 2D data on 3D plot
=======================

Demonstrates using ax.plot's zdir keyword to plot 2D data on
selective axes of a 3D plot.
"""

from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm


fig = plt.figure()
ax = fig.gca(projection='3d')
font_size = 18
# Plot a sin curve using the x and y axes.
x = np.array(parN)
y = np.array(LL)
z = np.array(robar2)
label = np.array(best[1:])
ax.plot(x, y, z, zdir='z', linewidth=4, markersize=12, marker='o', label='Best Model Path')

for label, x, y, z in zip(label, x, y, z):
    ax.text(x, y, z, label, fontsize=font_size)

x = [len(results[i].data.betaNames) for i in range(len(results))]
y = np.array([results[i].data.logLike for i in range(len(results))])
z = np.array([results[i].data.rhoBarSquare for i in range(len(results))])
label = np.array(ModNameSeqence)
ax.plot(x, y, z, '--', zdir='z', linewidth=2, markersize=12, marker='x', label='Model Testing')

for label, x, y, z in zip(label, x, y, z):
    ax.text(x, y, z, label, fontsize=font_size)
# Make legend, set axes limits and labels
ax.legend(fontsize=font_size)
# Make legend, set axes limits and labels
#ax.set_xlim(0, 1)
#ax.set_ylim(0, 1)
#ax.set_zlim(0, 1)
ax.set_xlabel('Number of Parameters', fontsize=font_size)
ax.set_ylabel('Log Likelihood', fontsize=font_size)
ax.set_zlabel('ro bar square', fontsize=font_size)

# Customize the view angle so it's easier to see that the scatter points lie
# on the plane y=0
#ax.view_init(elev=20., azim=-35)
plt.savefig('MLPanel.png', dpi=300)
plt.show()

In [ ]:
# Function that calculates the market shares for SM for a given scaling of the cost

def scenario(HEvalue, costScale, database, pickShare,  draws=100):
    '''
    ...#costScale includes the variability of cost
    ...#HEvalue the simulations on different HE values
    ...#To test HE effect in conditions similar to Train, 
    ...#we put SM_TT_SCALED = TRAIN_TT_SCALED
    '''
    share = {'CAR': 3,'TRAIN': 1,'SM': 2}
    scale = pandas['SM_HE_SCALED'][0]/pandas['SM_HE'][0]
    
    #Utility functions
    TRAIN_SP = ASC_TRAIN                                      + \
     BETA_COST_random             * (TRAIN_COST_SCALED * costScale)**q_COST + \
     BETA_TIME_TRAIN_random_Norm       * TRAIN_TT_SCALED**q_TIME   + \
     BETA_HE_random_Norm               * pandas['TRAIN_HE_SCALED'].min()**q_HE           + \
     B_AGE_ADULTS_TRAIN * AGE_2                     + \
     B_AGE_3_TRAIN      * AGE_3                     + \
     B_AGE_ADULTS_TRAIN * AGE_4                     + \
     B_AGE_5_TRAIN      * AGE_5                     + \
     B_LUGGAGE_TRAIN    * LUGGAGE                   + \
     B_MALE_TRAIN       * MALE                      + \
     B_GA_TRAIN         * GA                        + \
     B_BUSINESS_TRAIN   * BUSINESS                  + \
     B_PRIVATE          * PRIVATE                   + \
     B_COMMUTER         * COMMUTER
    
    
    SM_SP = ASC_SM                                   + \
         BETA_COST_random * (SM_COST_SCALED * costScale)**q_COST + \
         BETA_TIME_SM_random       * SM_TT_SCALED**q_TIME        + \
         BETA_HE_random_Norm            * pandas['SM_HE_SCALED'].min()**q_HE  + \
         B_AGE_ADULTS_SM * AGE_2                  + \
         B_AGE_ADULTS_SM * AGE_3                  + \
         B_AGE_4_SM      * AGE_4                  + \
         B_AGE_5_SM      * AGE_5                  + \
         B_FIRST_SM      * FIRST                  + \
         B_BUSINESS_SM   * BUSINESS               + \
         B_PRIVATE       * PRIVATE                + \
         B_COMMUTER      * COMMUTER
    '''
    TRAIN_SP = ASC_TRAIN                                      + \
     BETA_COST_random             * (TRAIN_COST_SCALED * costScale)**q_COST + \
     BETA_TIME_TRAIN_random_Norm       * TRAIN_TT_SCALED**q_TIME   + \
     BETA_HE_random_Norm               * (HEvalue*scale)**q_HE           + \
     B_AGE_ADULTS_TRAIN * AGE_2                     + \
     B_AGE_3_TRAIN      * AGE_3                     + \
     B_AGE_ADULTS_TRAIN * AGE_4                     + \
     B_AGE_5_TRAIN      * AGE_5                     + \
     B_LUGGAGE_TRAIN    * LUGGAGE                   + \
     B_MALE_TRAIN       * MALE                      + \
     B_GA_TRAIN         * GA                        + \
     B_BUSINESS_TRAIN   * BUSINESS                  + \
     B_PRIVATE          * PRIVATE                   + \
     B_COMMUTER         * COMMUTER
    
    
    SM_SP = ASC_SM                                   + \
         BETA_COST_random * (SM_COST_SCALED * costScale)**q_COST + \
         BETA_TIME_SM_random       * SM_TT_SCALED**q_TIME        + \
         BETA_HE_random_Norm            * (HEvalue*scale)**q_HE  + \
         B_AGE_ADULTS_SM * AGE_2                  + \
         B_AGE_ADULTS_SM * AGE_3                  + \
         B_AGE_4_SM      * AGE_4                  + \
         B_AGE_5_SM      * AGE_5                  + \
         B_FIRST_SM      * FIRST                  + \
         B_BUSINESS_SM   * BUSINESS               + \
         B_PRIVATE       * PRIVATE                + \
         B_COMMUTER      * COMMUTER
    '''
    V = {3: Car_SP,1: TRAIN_SP,2: SM_SP}
    av = {3: CAR_AV_SP,1: TRAIN_AV_SP,2: SM_AV}
    
    #Lets Look at CARs market share
    prob_SM = models.logit(V,av,share[pickShare])
    
    MCprob = MonteCarlo(prob_SM)
    
    simulate = {'Prob. SM': MCprob}
    
    biosim  = bio.BIOGEME(database,simulate,numberOfDraws=draws)
    
    biosim.modelName = f"marketShares_SM_HE_"+pickShare+"_NO_PANEL"
    
    simresults = biosim.simulate(results[-1].data.betaValues)
    
    return simresults['Prob. SM'].mean()

In [ ]:
# Duplicate the database
#database_young = db.Database("young",pandas[pandas['AGE'] == 1].copy())
database_middelage = db.Database("middelage",pandas[(pandas['AGE'] == 1)|(pandas['AGE'] == 2)|(pandas['AGE'] == 3)].copy())
database_senior = db.Database("senior",pandas[(pandas['AGE'] == 4)|(pandas['AGE'] == 5)].copy())


print(f"Total number of observations: {database.getNumberOfObservations()}")
#print(f"Young                  : {database_young.getNumberOfObservations()}")
print(f"Middle age               : {database_middelage.getNumberOfObservations()}")
print(f"Senior                 : {database_senior.getNumberOfObservations()}")

scales = np.arange(0.8,1.2,0.05)
HEs = np.arange(5,45,5)

shares_mid = [scenario(hhe,s,database_middelage,'CAR') for hhe in HEs for s in scales]
shares_sen = [scenario(hhe,s,database_senior,'CAR') for hhe in HEs for s in scales]

In [ ]:
# Duplicate the database
#database_young = db.Database("young",pandas[pandas['AGE'] == 1].copy())
database_Commuter = db.Database("commuters",pandas[(pandas['PURPOSE'] == 1)].copy())
database_Business = db.Database("business",pandas[(pandas['PURPOSE'] == 3)].copy())
database_Others = db.Database("business",pandas[(pandas['PURPOSE'] == 2)|(pandas['PURPOSE'] == 4)].copy())


print(f"Total number of observations: {database.getNumberOfObservations()}")
#print(f"Young                  : {database_young.getNumberOfObservations()}")
print(f"Commuter               : {database_Commuter.getNumberOfObservations()}")
print(f"Business                 : {database_Business.getNumberOfObservations()}")
print(f"Others                 : {database_Others.getNumberOfObservations()}")

scales = np.arange(0.8,1.2,0.05)
HEs = np.arange(5,45,5)

shares_Commuter = [scenario(hhe,s,database_Commuter,'CAR') for hhe in HEs for s in scales]
shares_Business = [scenario(hhe,s,database_Business,'CAR') for hhe in HEs for s in scales]
shares_Others = [scenario(hhe,s,database_Others,'CAR') for hhe in HEs for s in scales]

In [ ]:
database_ALL = db.Database("ALL",pandas.copy())
shares_ALL = [scenario(hhe,s,database_ALL,'CAR') for hhe in HEs for s in scales]

In [ ]:
# Function that calculates the market shares for SM for a given scaling of the cost

def scenarioHEmM(costScale, database, pickShare,  draws=100, HEmin=True):
    '''
    ...#costScale includes the variability of cost
    ...#HEvalue the simulations on different HE values
    ...#To test HE effect in conditions similar to Train, 
    ...#we put SM_TT_SCALED = TRAIN_TT_SCALED
    '''
    share = {'CAR': 3,'TRAIN': 1,'SM': 2}
    scale = pandas['SM_HE_SCALED'][0]/pandas['SM_HE'][0]
    if HEmin:
        tr_HE_tt_SIM = pandas['TRAIN_HE_SCALED'].min()
        sm_HE_tt_SIM = pandas['SM_HE_SCALED'].min()
    else:
        tr_HE_tt_SIM = pandas['TRAIN_HE_SCALED'].max()
        sm_HE_tt_SIM = pandas['SM_HE_SCALED'].max()
    
    #Utility functions
    TRAIN_SP = ASC_TRAIN                                      + \
         BETA_COST_random             * (TRAIN_COST_SCALED * costScale)**q_COST + \
         BETA_TIME_TRAIN_random_Norm       * TRAIN_TT_SCALED**q_TIME   + \
         BETA_HE_random_Norm               * tr_HE_tt_SIM**q_HE           + \
         B_AGE_ADULTS_TRAIN * AGE_2                     + \
         B_AGE_3_TRAIN      * AGE_3                     + \
         B_AGE_ADULTS_TRAIN * AGE_4                     + \
         B_AGE_5_TRAIN      * AGE_5                     + \
         B_LUGGAGE_TRAIN    * LUGGAGE                   + \
         B_MALE_TRAIN       * MALE                      + \
         B_GA_TRAIN         * GA                        + \
         B_BUSINESS_TRAIN   * BUSINESS                  + \
         B_PRIVATE          * PRIVATE                   + \
         B_COMMUTER         * COMMUTER
    
    
    SM_SP = ASC_SM                                   + \
         BETA_COST_random * (SM_COST_SCALED * costScale)**q_COST + \
         BETA_TIME_SM_random       * SM_TT_SCALED**q_TIME        + \
         BETA_HE_random_Norm            * sm_HE_tt_SIM**q_HE  + \
         B_AGE_ADULTS_SM * AGE_2                  + \
         B_AGE_ADULTS_SM * AGE_3                  + \
         B_AGE_4_SM      * AGE_4                  + \
         B_AGE_5_SM      * AGE_5                  + \
         B_FIRST_SM      * FIRST                  + \
         B_BUSINESS_SM   * BUSINESS               + \
         B_PRIVATE       * PRIVATE                + \
         B_COMMUTER      * COMMUTER
    '''
    TRAIN_SP = ASC_TRAIN                                      + \
     BETA_COST_random             * (TRAIN_COST_SCALED * costScale)**q_COST + \
     BETA_TIME_TRAIN_random_Norm       * TRAIN_TT_SCALED**q_TIME   + \
     BETA_HE_random_Norm               * (HEvalue*scale)**q_HE           + \
     B_AGE_ADULTS_TRAIN * AGE_2                     + \
     B_AGE_3_TRAIN      * AGE_3                     + \
     B_AGE_ADULTS_TRAIN * AGE_4                     + \
     B_AGE_5_TRAIN      * AGE_5                     + \
     B_LUGGAGE_TRAIN    * LUGGAGE                   + \
     B_MALE_TRAIN       * MALE                      + \
     B_GA_TRAIN         * GA                        + \
     B_BUSINESS_TRAIN   * BUSINESS                  + \
     B_PRIVATE          * PRIVATE                   + \
     B_COMMUTER         * COMMUTER
    
    
    SM_SP = ASC_SM                                   + \
         BETA_COST_random * (SM_COST_SCALED * costScale)**q_COST + \
         BETA_TIME_SM_random       * SM_TT_SCALED**q_TIME        + \
         BETA_HE_random_Norm            * (HEvalue*scale)**q_HE  + \
         B_AGE_ADULTS_SM * AGE_2                  + \
         B_AGE_ADULTS_SM * AGE_3                  + \
         B_AGE_4_SM      * AGE_4                  + \
         B_AGE_5_SM      * AGE_5                  + \
         B_FIRST_SM      * FIRST                  + \
         B_BUSINESS_SM   * BUSINESS               + \
         B_PRIVATE       * PRIVATE                + \
         B_COMMUTER      * COMMUTER
    '''
    V = {3: Car_SP,1: TRAIN_SP,2: SM_SP}
    av = {3: CAR_AV_SP,1: TRAIN_AV_SP,2: SM_AV}
    
    #Lets Look at CARs market share
    prob_SM = models.logit(V,av,share[pickShare])
    
    MCprob = MonteCarlo(prob_SM)
    
    simulate = {'Prob. SM': MCprob}
    
    biosim  = bio.BIOGEME(database,simulate,numberOfDraws=draws)
    
    biosim.modelName = f"marketShares_SM_HE_"+pickShare+"_NO_PANEL"
    
    simresults = biosim.simulate(results[-1].data.betaValues)
    
    return simresults['Prob. SM'].mean()

In [ ]:
scales = np.arange(0.8,1.2,0.05)
shares_Commuter_m = [scenarioHEmM(s,database_Commuter,'CAR',HEmin=True) for s in scales]
shares_Business_m = [scenarioHEmM(s,database_Business,'CAR',HEmin=True) for s in scales]
shares_Others_m = [scenarioHEmM(s,database_Others,'CAR',HEmin=True) for s in scales]
shares_mid_m = [scenarioHEmM(s,database_middelage,'CAR',HEmin=True) for s in scales]
shares_sen_m = [scenarioHEmM(s,database_senior,'CAR',HEmin=True) for s in scales]
shares_ALL_m = [scenarioHEmM(s,database_ALL,'CAR',HEmin=True) for s in scales]

In [ ]:
shares_Commuter_M = [scenarioHEmM(s,database_Commuter,'CAR',HEmin=False) for s in scales]
shares_Business_M = [scenarioHEmM(s,database_Business,'CAR',HEmin=False) for s in scales]
shares_Others_M = [scenarioHEmM(s,database_Others,'CAR',HEmin=False) for s in scales]
shares_mid_M = [scenarioHEmM(s,database_middelage,'CAR',HEmin=False) for s in scales]
shares_sen_M = [scenarioHEmM(s,database_senior,'CAR',HEmin=False) for s in scales]
shares_ALL_M = [scenarioHEmM(s,database_ALL,'CAR',HEmin=False) for s in scales]

In [ ]:
# plot the data
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
ax.plot(scales, shares_ALL_m, c='g', label='(FULL population) constant HE train='+str(100*pandas['TRAIN_HE_SCALED'].min())+'min and HE SM='+str(100*pandas['SM_HE_SCALED'].min())+'min')
ax.plot(scales, shares_ALL_M, c='r', label='(FULL population) constant HE train='+str(100*pandas['TRAIN_HE_SCALED'].max())+'min and HE SM='+str(int(100*pandas['SM_HE_SCALED'].max()))+'min')
ax.plot(scales, CAR_Market_share*np.ones(len(shares_ALL_m),), c='b', label='(FULL population) market share as from survey results')
plt.axvline(x=1, c='black', label='Cost 100% as from survey results')

ax.set_title('IMPACT of Public Transportation policies ON CAR market share \n HE and cost policy bonduaries for SM and Train \n Effects on full population')
ax.legend(fontsize=12)
ax.set_xlabel('Cost variation on both Public Transport (Train and SM: the actual cost 1=100%)', fontsize=font_size)
ax.set_ylabel('CAR Market Share', fontsize=font_size)
plt.savefig('CARmsImpact-FULLPOP.png', dpi=300)
# display the plot
plt.show()

In [ ]:
senior = pandas['CHOICE'][(pandas['CHOICE']==3) & ((pandas['AGE'] == 4)|(pandas['AGE'] == 5))].count()/pandas['CHOICE'][((pandas['AGE'] == 4)|(pandas['AGE'] == 5))].count()
NON_senior = pandas['CHOICE'][(pandas['CHOICE']==3) & ((pandas['AGE'] == 1)|(pandas['AGE'] == 2)|(pandas['AGE'] == 3))].count()/pandas['CHOICE'][((pandas['AGE'] == 1)|(pandas['AGE'] == 2)|(pandas['AGE'] == 3))].count()
# plot the data
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
ax.plot(scales, np.array(shares_sen_m), c='g', label='(senior) constant HE train='+str(100*pandas['TRAIN_HE_SCALED'].min())+'min and HE SM='+str(100*pandas['SM_HE_SCALED'].min())+'min')
ax.plot(scales, np.array(shares_sen_M), c='r', label='(senior) constant HE train='+str(100*pandas['TRAIN_HE_SCALED'].max())+'min and HE SM='+str(100*pandas['SM_HE_SCALED'].max())+'min')
ax.plot(scales, senior*np.ones(len(shares_ALL_m),), c='b', label='(senior) market share as from survey results')

ax.plot(scales, np.array(shares_mid_m), '--', c='g', label='(non senior) constant HE train='+str(100*pandas['TRAIN_HE_SCALED'].min())+'min and HE SM='+str(100*pandas['SM_HE_SCALED'].min())+'min')
ax.plot(scales, np.array(shares_mid_M), '--', c='r', label='(non senior) constant HE train='+str(100*pandas['TRAIN_HE_SCALED'].max())+'min and HE SM='+str(100*pandas['SM_HE_SCALED'].max())+'min')
ax.plot(scales, NON_senior*np.ones(len(shares_ALL_m),),'--', c='b', label='(non senior) market share as from survey results')



plt.axvline(x=1, c='black', label='cost 100% as from survey results')

ax.set_title('IMPACT of Public Transportation policies ON CAR market share \n HE and cost policy bonduaries for SM and Train \n effects on Seniors vs Others')
ax.legend(fontsize=12)
ax.set_xlabel('Cost variation on both Public Transport (Train and SM: the actual cost 1=100%)', fontsize=font_size)
ax.set_ylabel('CAR Market Share', fontsize=font_size)
# display the plot
plt.savefig('CARmsImpact-AGE.png', dpi=300)
plt.show()

In [ ]:
Business_MS = pandas['CHOICE'][(pandas['CHOICE']==3) & (pandas['PURPOSE'] == 3)].count()/pandas['CHOICE'][(pandas['PURPOSE'] == 3)].count()
Commuter_MS = pandas['CHOICE'][(pandas['CHOICE']==3) & (pandas['PURPOSE'] == 1)].count()/pandas['CHOICE'][(pandas['PURPOSE'] == 1)].count()
Others_MS = pandas['CHOICE'][(pandas['CHOICE']==3) & ((pandas['PURPOSE'] == 2)|(pandas['PURPOSE'] == 4))].count()/pandas['CHOICE'][((pandas['PURPOSE'] == 2)|(pandas['PURPOSE'] == 4))].count()
# plot the data
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
ax.plot(scales, shares_Commuter_m, c='g', label='(Commuter) constant HE train='+str(100*pandas['TRAIN_HE_SCALED'].min())+'min and HE SM='+str(100*pandas['SM_HE_SCALED'].min())+'min')
ax.plot(scales, shares_Commuter_M, c='r', label='(Commuter) constant HE train='+str(100*pandas['TRAIN_HE_SCALED'].max())+'min and HE SM='+str(100*pandas['SM_HE_SCALED'].max())+'min')
ax.plot(scales, Commuter_MS*np.ones(len(shares_ALL_m),), c='b', label='(Commuter) market share as from survey results')


ax.plot(scales, shares_Business_m, '--', c='g', label='(Business) constant HE train='+str(100*pandas['TRAIN_HE_SCALED'].min())+'min and HE SM='+str(100*pandas['SM_HE_SCALED'].min())+'min')
ax.plot(scales, shares_Business_M, '--', c='r', label='(Business) constant HE train='+str(100*pandas['TRAIN_HE_SCALED'].max())+'min and HE SM='+str(100*pandas['SM_HE_SCALED'].max())+'min')
ax.plot(scales, Business_MS*np.ones(len(shares_ALL_m),), '--', c='b', label='(Business) market share as from survey results')

ax.plot(scales, shares_Others_m, '*-', c='g', label='(Others) constant HE train='+str(100*pandas['TRAIN_HE_SCALED'].min())+'min and HE SM='+str(100*pandas['SM_HE_SCALED'].min())+'min')
ax.plot(scales, shares_Others_M, '*-', c='r', label='(Others) constant HE train='+str(100*pandas['TRAIN_HE_SCALED'].max())+'min and HE SM='+str(100*pandas['SM_HE_SCALED'].max())+'min')
ax.plot(scales, Others_MS*np.ones(len(shares_ALL_m),), '*-', c='b', label='(Others) market share as from survey results')


plt.axvline(x=1, c='black', label='cost 100% as from survey results')

ax.set_title('IMPACT of Public Transportation policies ON CAR market share \n HE and cost policy bonduaries for SM and Train \n Effects on Business Vs. Commuters')
ax.legend(fontsize=12)
ax.set_xlabel('Cost variation on both Public Transport (Train and SM: the actual cost 1=100%)', fontsize=font_size)
ax.set_ylabel('CAR Market Share', fontsize=font_size)
# display the plot
plt.savefig('CARmsImpact-PURPOSE.png', dpi=300)
plt.show()

In [ ]:
CAR_Market_share = pandas['CHOICE'][(pandas['CHOICE']==3)].count()/pandas['CHOICE'].count()
print(CAR_Market_share)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

X, Y = np.meshgrid(HEs, scales)
zC = np.array(shares_Commuter)
zB =np.array(shares_Business)
zO =np.array(shares_Others)
zAll =np.array(shares_ALL)

ZC = zC.reshape(X.shape)
ZB = zB.reshape(X.shape)
ZO = zO.reshape(X.shape)
ZAll = zAll.reshape(X.shape)
ZCar0=CAR_Market_share*np.ones(X.shape)

ax.plot_surface(X, Y, ZAll,cmap=cm.coolwarm, rstride=8, cstride=8, alpha=0.01, label='Commuters')
cset = ax.contourf(X, Y, ZAll, zdir='z', offset=0.275, cmap=cm.coolwarm)
cset = ax.contourf(X, Y, ZAll, zdir='x', offset=0,cmap=cm.coolwarm)
cset = ax.contourf(X, Y, ZAll, zdir='y', offset=1.15, cmap=cm.coolwarm)

ax.plot_surface(X, Y, ZCar0,cmap=cm.coolwarm, rstride=8, cstride=8, alpha=0.3, label='Commuters')
#cset = ax.contourf(X, Y, ZCar0, zdir='z', offset=0.275, cmap=cm.coolwarm)
cset = ax.contourf(X, Y, ZCar0, zdir='x', offset=0,cmap=cm.coolwarm)
cset = ax.contourf(X, Y, ZCar0, zdir='y', offset=1.15, cmap=cm.coolwarm)

#ax.plot_surface(X, Y, ZC,cmap=cm.coolwarm, rstride=8, cstride=8, alpha=0.3, label='Commuters')
#cset = ax.contourf(X, Y, ZC, zdir='z', offset=0.24, cmap=cm.coolwarm)
#cset = ax.contourf(X, Y, ZC, zdir='x', offset=0,cmap=cm.coolwarm)
#cset = ax.contourf(X, Y, ZC, zdir='y', offset=1.15, cmap=cm.coolwarm)

#ax.plot_surface(X, Y, ZB, cmap=cm.coolwarm, rstride=8, cstride=8, alpha=0.3, label='Business')
#cset = ax.contourf(X, Y, ZB, zdir='z', offset=0.23, cmap=cm.coolwarm)
#cset = ax.contourf(X, Y, ZB, zdir='x', offset=1,cmap=cm.coolwarm)
#cset = ax.contourf(X, Y, ZB, zdir='y', offset=1.2, cmap=cm.coolwarm)

#ax.plot_surface(X, Y, ZO, cmap=cm.coolwarm, rstride=8, cstride=8, alpha=0.3, label='Others')
#cset = ax.contourf(X, Y, ZO, zdir='z', offset=0.23, cmap=cm.coolwarm)
#cset = ax.contourf(X, Y, ZO, zdir='x', offset=1,cmap=cm.coolwarm)
#cset = ax.contourf(X, Y, ZO, zdir='y', offset=1.2, cmap=cm.coolwarm)

ax.set_xlabel('TRAIN HE minutes')
#ax.set_xlim(0, 45)
ax.set_ylabel('TRAIN Cost')
#ax.set_ylim(0.8, 1.15)
ax.set_zlabel('CAR Market Shares')
#ax.set_zlim(0.24, 0.32)
#plt.savefig('MARKETshareCarPURPOSE-SMtrainEqualTT.png', dpi=300)
plt.show()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

X, Y = np.meshgrid(HEs, scales)
zm = np.array(shares_mid)
zs =np.array(shares_sen)
Zm = zm.reshape(X.shape)
Zs = zs.reshape(X.shape)

ax.plot_surface(X, Y, Zs,cmap=cm.coolwarm, rstride=8, cstride=8, alpha=0.1, label='senior')
cset = ax.contourf(X, Y, Zs, zdir='z', offset=0.355, cmap=cm.coolwarm)
cset = ax.contourf(X, Y, Zs, zdir='x', offset=1,cmap=cm.coolwarm)
cset = ax.contourf(X, Y, Zs, zdir='y',offset= 1.18, cmap=cm.coolwarm)

ax.plot_surface(X, Y, Zm, cmap=cm.coolwarm, rstride=8, cstride=8, alpha=0.1, label='middel age and young')
cset = ax.contourf(X, Y, Zm, zdir='z', offset=0.355, cmap=cm.coolwarm)
cset = ax.contourf(X, Y, Zm, zdir='x', offset=1, cmap=cm.coolwarm)
cset = ax.contourf(X, Y, Zm, zdir='y',offset= 1.18, cmap=cm.coolwarm)

ax.set_xlabel('SM HE minutes')
ax.set_ylabel('SM Cost')
ax.set_zlabel('CAR Market Shares')
#plt.savefig('MARKETshareCarSeniorYoung-SMtrainEqualTT.png', dpi=300)
plt.show()